In [1]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sklearn

sklearn.__version__

'1.0.2'

In [3]:
!pip install six

In [4]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [5]:
import pandas as pd
data_orig = pd.read_excel('drive/My Drive/Colab Notebooks/WisconsinDataset/Forms_cleaned.xlsx', 'form1')

In [6]:
data_orig.tail()

,EID,FormID,Flagged,Attempt,tot_time,iresp.1,iresp.2,iresp.3,iresp.4,iresp.5,iresp.6,iresp.7,iresp.8,iresp.9,iresp.10,iresp.11,iresp.12,iresp.13,iresp.14,iresp.15,iresp.16,iresp.17,iresp.18,iresp.19,iresp.20,iresp.21,iresp.22,iresp.23,iresp.24,iresp.25,iresp.26,iresp.27,iresp.28,iresp.29,iresp.30,iresp.31,iresp.32,iresp.33,iresp.34,iresp.35,...,idur.131,idur.132,idur.133,idur.134,idur.135,idur.136,idur.137,idur.138,idur.139,idur.140,idur.141,idur.142,idur.143,idur.144,idur.145,idur.146,idur.147,idur.148,idur.149,idur.150,idur.151,idur.152,idur.153,idur.154,idur.155,idur.156,idur.157,idur.158,idur.159,idur.160,idur.161,idur.162,idur.163,idur.164,idur.165,idur.166,idur.167,idur.168,idur.169,idur.170
1631,e101632,form1,0,1,8760,1,1,3.0,3,2,2.0,3,1.0,4,1,4,4.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,4.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,43,59,61,70,24,67,46,41,86,47,54,24,46,36,68,39,66,33,34,13,38,35,37,23,64,20,19,72,63,15,10,72,40,10,10,52,56,47,61,19
1632,e101633,form1,0,1,10568,1,1,3.0,3,2,2.0,3,1.0,1,1,4,3.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,62,42,22,26,35,44,69,69,92,79,79,18,17,48,230,58,44,90,60,18,74,49,149,14,28,38,27,128,52,8,11,116,45,11,7,116,79,59,48,40
1633,e101634,form1,0,1,12449,1,1,3.0,3,2,2.0,3,1.0,3,1,4,1.0,4,4.0,3,1.0,3,1.0,1,1,2.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,165,31,52,35,27,80,47,59,78,90,137,42,35,35,52,109,75,114,36,33,34,168,62,68,44,18,36,144,98,11,47,164,28,11,30,54,19,27,43,14
1634,e101635,form1,0,1,8209,1,1,3.0,3,2,2.0,3,1.0,4,1,4,3.0,2,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,42,36,12,33,26,57,49,38,123,39,90,13,41,38,17,36,41,30,26,32,50,18,69,45,30,24,27,50,24,43,14,45,32,14,28,17,18,109,94,14
1635,e101636,form1,0,1,9311,1,1,3.0,3,2,2.0,3,4.0,4,1,1,3.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,52,25,86,72,28,39,70,62,156,45,36,22,24,34,14,30,72,18,44,17,61,14,61,23,46,21,35,61,38,14,12,61,45,13,14,62,28,51,45,46


In [7]:
data_final = data_orig
data_final

,EID,FormID,Flagged,Attempt,tot_time,iresp.1,iresp.2,iresp.3,iresp.4,iresp.5,iresp.6,iresp.7,iresp.8,iresp.9,iresp.10,iresp.11,iresp.12,iresp.13,iresp.14,iresp.15,iresp.16,iresp.17,iresp.18,iresp.19,iresp.20,iresp.21,iresp.22,iresp.23,iresp.24,iresp.25,iresp.26,iresp.27,iresp.28,iresp.29,iresp.30,iresp.31,iresp.32,iresp.33,iresp.34,iresp.35,...,idur.131,idur.132,idur.133,idur.134,idur.135,idur.136,idur.137,idur.138,idur.139,idur.140,idur.141,idur.142,idur.143,idur.144,idur.145,idur.146,idur.147,idur.148,idur.149,idur.150,idur.151,idur.152,idur.153,idur.154,idur.155,idur.156,idur.157,idur.158,idur.159,idur.160,idur.161,idur.162,idur.163,idur.164,idur.165,idur.166,idur.167,idur.168,idur.169,idur.170
0,e100001,form1,0,5,10133,1,1,3.0,1,2,1.0,4,4.0,1,1,3,1.0,2,2.0,4,1.0,2,1.0,4,3,3.0,3.0,4,3,3.0,1.0,2.0,1.0,1.0,2.0,4.0,2,1,2.0,1.0,...,85,47,60,46,98,106,92,63,103,48,42,66,26,57,14,63,45,28,21,15,64,23,61,28,35,28,64,122,20,18,17,102,55,50,17,28,53,82,100,21
1,e100002,form1,1,1,12409,1,3,3.0,1,1,3.0,4,4.0,2,4,3,2.0,1,2.0,3,4.0,2,2.0,3,3,3.0,3.0,3,1,3.0,2.0,4.0,1.0,4.0,4.0,2.0,1,1,4.0,1.0,...,191,48,52,108,69,59,93,133,136,94,139,43,23,82,56,32,77,87,75,34,98,30,107,42,126,31,47,51,83,64,22,121,114,44,36,53,120,93,87,29
2,e100003,form1,1,1,12457,2,3,4.0,4,2,3.0,3,4.0,2,3,1,3.0,4,4.0,4,1.0,2,1.0,3,2,4.0,3.0,4,3,4.0,1.0,2.0,4.0,3.0,3.0,1.0,4,4,1.0,3.0,...,62,54,89,72,68,78,77,77,170,48,84,62,52,122,41,98,38,49,52,27,62,18,78,47,109,47,44,72,23,16,73,34,92,83,50,30,63,96,79,48
3,e100004,form1,1,3,11389,1,1,1.0,4,2,1.0,4,4.0,4,4,3,4.0,4,1.0,4,2.0,2,3.0,3,2,2.0,3.0,3,1,1.0,1.0,2.0,3.0,2.0,2.0,4.0,3,2,4.0,3.0,...,79,69,105,75,60,64,77,54,73,50,60,29,37,72,34,78,74,93,59,39,39,30,34,35,62,33,47,87,83,45,30,93,49,26,57,41,59,41,84,17
4,e100005,form1,1,5,13669,1,1,2.0,4,2,2.0,1,4.0,2,1,4,NaN,2,1.0,4,4.0,1,2.0,3,3,3.0,3.0,3,4,4.0,1.0,NaN,1.0,1.0,NaN,3.0,2,4,4.0,1.0,...,171,29,92,56,55,189,146,108,151,71,57,31,33,138,131,92,110,30,57,20,34,34,72,56,96,46,119,66,133,34,11,0,58,127,63,68,164,74,63,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,e101632,form1,0,1,8760,1,1,3.0,3,2,2.0,3,1.0,4,1,4,4.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,4.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,43,59,61,70,24,67,46,41,86,47,54,24,46,36,68,39,66,33,34,13,38,35,37,23,64,20,19,72,63,15,10,72,40,10,10,52,56,47,61,19
1632,e101633,form1,0,1,10568,1,1,3.0,3,2,2.0,3,1.0,1,1,4,3.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,62,42,22,26,35,44,69,69,92,79,79,18,17,48,230,58,44,90,60,18,74,49,149,14,28,38,27,128,52,8,11,116,45,11,7,116,79,59,48,40
1633,e101634,form1,0,1,12449,1,1,3.0,3,2,2.0,3,1.0,3,1,4,1.0,4,4.0,3,1.0,3,1.0,1,1,2.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,165,31,52,35,27,80,47,59,78,90,137,42,35,35,52,109,75,114,36,33,34,168,62,68,44,18,36,144,98,11,47,164,28,11,30,54,19,27,43,14
1634,e101635,form1,0,1,8209,1,1,3.0,3,2,2.0,3,1.0,4,1,4,3.0,2,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,42,36,12,33,26,57,49,38,123,39,90,13,41,38,17,36,41,30,26,32,50,18,69,45,30,24,27,50,24,43,14,45,32,14,28,17,18,109,94,14


In [8]:
print(data_final.isnull().sum())
print(data_final.shape)

EID         0
FormID      0
Flagged     0
Attempt     0
tot_time    0
           ..
idur.166    0
idur.167    0
idur.168    0
idur.169    0
idur.170    0
Length: 515, dtype: int64
(1636, 515)


In [9]:
##### time #####
data_final['mean_idur'] = data_final.loc[:, 'idur.1':'idur.170'].mean(axis=1)
data_final['median_idur'] = data_final.loc[:, 'idur.1':'idur.170'].median(axis=1)
data_final['max_idur'] = data_final.loc[:, 'idur.1':'idur.170'].max(axis=1)
data_final['min_idur'] = data_final.loc[:, 'idur.1':'idur.170'].min(axis=1)

#### correct answers####
data_final['iraw_score'] = data_final.loc[:, 'iraw.1':'iraw.170'].sum(axis=1)
data_final['iraw_score_pct'] = data_final['iraw_score']/170
data_final

,EID,FormID,Flagged,Attempt,tot_time,iresp.1,iresp.2,iresp.3,iresp.4,iresp.5,iresp.6,iresp.7,iresp.8,iresp.9,iresp.10,iresp.11,iresp.12,iresp.13,iresp.14,iresp.15,iresp.16,iresp.17,iresp.18,iresp.19,iresp.20,iresp.21,iresp.22,iresp.23,iresp.24,iresp.25,iresp.26,iresp.27,iresp.28,iresp.29,iresp.30,iresp.31,iresp.32,iresp.33,iresp.34,iresp.35,...,idur.137,idur.138,idur.139,idur.140,idur.141,idur.142,idur.143,idur.144,idur.145,idur.146,idur.147,idur.148,idur.149,idur.150,idur.151,idur.152,idur.153,idur.154,idur.155,idur.156,idur.157,idur.158,idur.159,idur.160,idur.161,idur.162,idur.163,idur.164,idur.165,idur.166,idur.167,idur.168,idur.169,idur.170,mean_idur,median_idur,max_idur,min_idur,iraw_score,iraw_score_pct
0,e100001,form1,0,5,10133,1,1,3.0,1,2,1.0,4,4.0,1,1,3,1.0,2,2.0,4,1.0,2,1.0,4,3,3.0,3.0,4,3,3.0,1.0,2.0,1.0,1.0,2.0,4.0,2,1,2.0,1.0,...,92,63,103,48,42,66,26,57,14,63,45,28,21,15,64,23,61,28,35,28,64,122,20,18,17,102,55,50,17,28,53,82,100,21,56.323529,50.0,143,14,54,0.317647
1,e100002,form1,1,1,12409,1,3,3.0,1,1,3.0,4,4.0,2,4,3,2.0,1,2.0,3,4.0,2,2.0,3,3,3.0,3.0,3,1,3.0,2.0,4.0,1.0,4.0,4.0,2.0,1,1,4.0,1.0,...,93,133,136,94,139,43,23,82,56,32,77,87,75,34,98,30,107,42,126,31,47,51,83,64,22,121,114,44,36,53,120,93,87,29,69.158824,63.0,202,11,55,0.323529
2,e100003,form1,1,1,12457,2,3,4.0,4,2,3.0,3,4.0,2,3,1,3.0,4,4.0,4,1.0,2,1.0,3,2,4.0,3.0,4,3,4.0,1.0,2.0,4.0,3.0,3.0,1.0,4,4,1.0,3.0,...,77,77,170,48,84,62,52,122,41,98,38,49,52,27,62,18,78,47,109,47,44,72,23,16,73,34,92,83,50,30,63,96,79,48,69.547059,65.5,179,16,61,0.358824
3,e100004,form1,1,3,11389,1,1,1.0,4,2,1.0,4,4.0,4,4,3,4.0,4,1.0,4,2.0,2,3.0,3,2,2.0,3.0,3,1,1.0,1.0,2.0,3.0,2.0,2.0,4.0,3,2,4.0,3.0,...,77,54,73,50,60,29,37,72,34,78,74,93,59,39,39,30,34,35,62,33,47,87,83,45,30,93,49,26,57,41,59,41,84,17,62.882353,62.0,154,12,61,0.358824
4,e100005,form1,1,5,13669,1,1,2.0,4,2,2.0,1,4.0,2,1,4,NaN,2,1.0,4,4.0,1,2.0,3,3,3.0,3.0,3,4,4.0,1.0,NaN,1.0,1.0,NaN,3.0,2,4,4.0,1.0,...,146,108,151,71,57,31,33,138,131,92,110,30,57,20,34,34,72,56,96,46,119,66,133,34,11,0,58,127,63,68,164,74,63,17,76.547059,69.0,189,0,62,0.364706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,e101632,form1,0,1,8760,1,1,3.0,3,2,2.0,3,1.0,4,1,4,4.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,4.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,46,41,86,47,54,24,46,36,68,39,66,33,34,13,38,35,37,23,64,20,19,72,63,15,10,72,40,10,10,52,56,47,61,19,48.658824,43.0,152,8,156,0.917647
1632,e101633,form1,0,1,10568,1,1,3.0,3,2,2.0,3,1.0,1,1,4,3.0,4,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,69,69,92,79,79,18,17,48,230,58,44,90,60,18,74,49,149,14,28,38,27,128,52,8,11,116,45,11,7,116,79,59,48,40,59.335294,49.0,230,7,153,0.900000
1633,e101634,form1,0,1,12449,1,1,3.0,3,2,2.0,3,1.0,3,1,4,1.0,4,4.0,3,1.0,3,1.0,1,1,2.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,47,59,78,90,137,42,35,35,52,109,75,114,36,33,34,168,62,68,44,18,36,144,98,11,47,164,28,11,30,54,19,27,43,14,69.894118,47.0,322,7,154,0.905882
1634,e101635,form1,0,1,8209,1,1,3.0,3,2,2.0,3,1.0,4,1,4,3.0,2,4.0,3,1.0,2,1.0,1,1,3.0,3.0,3,4,4.0,3.0,1.0,1.0,3.0,1.0,3.0,2,2,4.0,3.0,...,49,38,123,39,90,13,41,38,17,36,41,30,26,32,50,18,69,45,30,24,27,50,24,43,14,45,32,14,28,17,18,109,94,14,45.376471,34.0,258,10,152,0.894118


In [10]:
data_final.fillna(data_final.mean(), inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [11]:
######## iraw_idur_OEF (both+OEF) #############
col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct']

for x in range(1,171):
  col_features.append('idur.'+str(x))

for i in range(1,171):
  col_features.append('iraw.'+str(i))

X = data_final.loc[:, col_features]
Y = data_final.Flagged    

**End of Dataset Features selection!**

In [12]:
from collections import Counter
from matplotlib import pyplot
from numpy import where
counter = Counter(Y)
print(counter)

Counter({0: 1590, 1: 46})


In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

In [14]:
# Import Libraries
from sklearn.preprocessing import MinMaxScaler
num_cols = X.columns

# apply standardization on numerical features
for i in num_cols:
    # fit on training data column
    scale = MinMaxScaler(feature_range=(0, 1)).fit(X[[i]])

    # transform the training data column
    X[i] = scale.transform(X[[i]])
    
print("\n\nTraining Features After Scaling: ")
print("=================================\n")
X



Training Features After Scaling: 



,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,idur.1,idur.2,idur.3,idur.4,idur.5,idur.6,idur.7,idur.8,idur.9,idur.10,idur.11,idur.12,idur.13,idur.14,idur.15,idur.16,idur.17,idur.18,idur.19,idur.20,idur.21,idur.22,idur.23,idur.24,idur.25,idur.26,idur.27,idur.28,idur.29,idur.30,idur.31,idur.32,idur.33,...,iraw.131,iraw.132,iraw.133,iraw.134,iraw.135,iraw.136,iraw.137,iraw.138,iraw.139,iraw.140,iraw.141,iraw.142,iraw.143,iraw.144,iraw.145,iraw.146,iraw.147,iraw.148,iraw.149,iraw.150,iraw.151,iraw.152,iraw.153,iraw.154,iraw.155,iraw.156,iraw.157,iraw.158,iraw.159,iraw.160,iraw.161,iraw.162,iraw.163,iraw.164,iraw.165,iraw.166,iraw.167,iraw.168,iraw.169,iraw.170
0,1.0,0.251140,0.254294,0.246154,0.037668,0.482759,0.000000,0.169065,0.041812,0.114007,0.066667,0.057377,0.126761,0.146814,0.323276,0.144886,0.232092,0.100649,0.136253,0.032258,0.181661,0.133820,0.119741,0.076754,0.113065,0.114286,0.077647,0.180812,0.227074,0.040541,0.087935,0.491597,0.380645,0.139319,0.131673,0.270916,0.119741,0.173034,0.305556,0.059809,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.2,0.373250,0.379208,0.379487,0.090583,0.379310,0.009804,0.151079,0.081882,0.250814,0.112121,0.196721,0.161972,0.119114,0.366379,0.051136,0.372493,0.152597,0.131387,0.039427,0.129758,0.216545,0.165049,0.096491,0.218593,0.354286,0.070588,0.154982,0.048035,0.139640,0.077710,0.256303,0.206452,0.123839,0.206406,0.338645,0.116505,0.215730,0.166667,0.028708,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.2,0.375825,0.382986,0.405128,0.069955,0.551724,0.068627,0.201439,0.055749,0.179153,0.042424,0.200820,0.373239,0.077562,0.512931,0.113636,0.146132,0.113636,0.243309,0.093190,0.110727,0.094891,0.271845,0.140351,0.236181,0.297143,0.298824,0.313653,0.144105,0.162162,0.153374,0.474790,0.296774,0.266254,0.459075,0.362550,0.391586,0.242697,0.229167,0.155502,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.6,0.318526,0.318125,0.369231,0.047534,0.413793,0.068627,0.219424,0.247387,0.153094,0.112121,0.262295,0.246479,0.041551,0.228448,0.238636,0.257880,0.074675,0.218978,0.189964,0.171280,0.199513,0.171521,0.028509,0.125628,0.234286,0.171765,0.184502,0.056769,0.056306,0.094070,0.285714,0.212903,0.191950,0.309609,0.155378,0.077670,0.146067,0.166667,0.105263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
4,1.0,0.440850,0.451111,0.441026,0.078924,0.000000,0.078431,0.406475,0.139373,0.250814,0.178788,0.258197,0.193662,0.152355,0.155172,0.485795,0.303725,0.116883,0.000000,0.462366,0.057093,0.211679,0.239482,0.162281,0.228643,0.914286,0.211765,0.143911,0.183406,0.322072,0.051125,0.285714,0.258065,0.000000,0.252669,0.474104,0.000000,0.076404,0.531250,0.086124,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,0.2,0.177477,0.179700,0.174359,0.045740,0.275862,1.000000,0.118705,0.073171,0.120521,0.339394,0.241803,0.154930,0.052632,0.056034,0.039773,0.203438,0.050325,0.133820,0.050179,0.044983,0.077859,0.090615,0.052632,0.213568,0.171429,0.265882,0.129151,0.034934,0.094595,0.110429,0.138655,0.148387,0.436533,0.081851,0.131474,0.093851,0.051685,0.326389,0.066986,...,1.0,1.0,1.0

In [15]:
!pip install mlxtend  

In [16]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

from mlxtend.classifier import StackingClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [17]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


In [18]:

from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
from sklearn.metrics import auc

from numpy import sqrt
from numpy import argmax

In [19]:
###### SMOTE and RandomUnderSampler (for balancing data) ############

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


In [20]:
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [21]:
import xlwt 
from xlwt import Workbook 
  
wb = Workbook() 

In [22]:

k_fold = model_selection.KFold(n_splits = 4, random_state=25, shuffle=True)

full_trainset_x = np.array([])
full_trainset_y = np.array([])
full_testset_x = np.array([])
full_testset_y = np.array([])

full_testset_pred1 = np.array([])
full_testset_pred2 = np.array([])
full_testset_pred3 = np.array([])
full_testset_pred4 = np.array([])
full_testset_pred5 = np.array([])
full_testset_pred6 = np.array([])
full_testset_pred7 = np.array([])
full_testset_pred8 = np.array([])
full_testset_pred9 = np.array([])

full_testset_probs1 = np.array([])
full_testset_probs2 = np.array([])
full_testset_probs3 = np.array([])
full_testset_probs4 = np.array([])
full_testset_probs5 = np.array([])
full_testset_probs6 = np.array([])
full_testset_probs7 = np.array([])
full_testset_probs8 = np.array([])
full_testset_probs9 = np.array([])

nosmote_probs1 = np.array([])
nosmote_probs2 = np.array([])
nosmote_probs3 = np.array([])
nosmote_probs4 = np.array([])
nosmote_probs5 = np.array([])
nosmote_probs6 = np.array([])
nosmote_probs7 = np.array([])
nosmote_probs8 = np.array([])
nosmote_probs9 = np.array([])

nosmote_pred1 = np.array([])
nosmote_pred2 = np.array([])
nosmote_pred3 = np.array([])
nosmote_pred4 = np.array([])
nosmote_pred5 = np.array([])
nosmote_pred6 = np.array([])
nosmote_pred7 = np.array([])
nosmote_pred8 = np.array([])
nosmote_pred9 = np.array([])

precision1_1 = np.array([])
recall1_1 = np.array([])
f1_score1_1 = np.array([])
kappa1_1 = np.array([])
roc1_1 = np.array([])
fpr1_1 = np.array([])

precision1_2 = np.array([])
recall1_2 = np.array([])
f1_score1_2 = np.array([])
kappa1_2 = np.array([])
roc1_2 = np.array([])
fpr1_2 = np.array([])

precision1_3 = np.array([])
recall1_3 = np.array([])
f1_score1_3 = np.array([])
kappa1_3 = np.array([])
roc1_3 = np.array([])
fpr1_3 = np.array([])

precision1_4 = np.array([])
recall1_4 = np.array([])
f1_score1_4 = np.array([])
kappa1_4 = np.array([])
roc1_4 = np.array([])
fpr1_4 = np.array([])


precision2_1 = np.array([])
recall2_1 = np.array([])
f1_score2_1 = np.array([])
kappa2_1 = np.array([])
roc2_1 = np.array([])
fpr2_1 = np.array([])

precision2_2 = np.array([])
recall2_2 = np.array([])
f1_score2_2 = np.array([])
kappa2_2 = np.array([])
roc2_2 = np.array([])
fpr2_2 = np.array([])

precision2_3 = np.array([])
recall2_3 = np.array([])
f1_score2_3 = np.array([])
kappa2_3 = np.array([])
roc2_3 = np.array([])
fpr2_3 = np.array([])

precision2_4 = np.array([])
recall2_4 = np.array([])
f1_score2_4 = np.array([])
kappa2_4 = np.array([])
roc2_4 = np.array([])
fpr2_4 = np.array([])


precision3_1 = np.array([])
recall3_1 = np.array([])
f1_score3_1 = np.array([])
kappa3_1 = np.array([])
roc3_1 = np.array([])
fpr3_1 = np.array([])

precision3_2 = np.array([])
recall3_2 = np.array([])
f1_score3_2 = np.array([])
kappa3_2 = np.array([])
roc3_2 = np.array([])
fpr3_2 = np.array([])

precision3_3 = np.array([])
recall3_3 = np.array([])
f1_score3_3 = np.array([])
kappa3_3 = np.array([])
roc3_3 = np.array([])
fpr3_3 = np.array([])

precision3_4 = np.array([])
recall3_4 = np.array([])
f1_score3_4 = np.array([])
kappa3_4 = np.array([])
roc3_4 = np.array([])
fpr3_4 = np.array([])


precision4_1 = np.array([])
recall4_1 = np.array([])
f1_score4_1 = np.array([])
kappa4_1 = np.array([])
roc4_1 = np.array([])
fpr4_1 = np.array([])

precision4_2 = np.array([])
recall4_2 = np.array([])
f1_score4_2 = np.array([])
kappa4_2 = np.array([])
roc4_2 = np.array([])
fpr4_2 = np.array([])

precision4_3 = np.array([])
recall4_3 = np.array([])
f1_score4_3 = np.array([])
kappa4_3 = np.array([])
roc4_3 = np.array([])
fpr4_3 = np.array([])

precision4_4 = np.array([])
recall4_4 = np.array([])
f1_score4_4 = np.array([])
kappa4_4 = np.array([])
roc4_4 = np.array([])
fpr4_4 = np.array([])


precision5_1 = np.array([])
recall5_1 = np.array([])
f1_score5_1 = np.array([])
kappa5_1 = np.array([])
roc5_1 = np.array([])
fpr5_1 = np.array([])

precision5_2 = np.array([])
recall5_2 = np.array([])
f1_score5_2 = np.array([])
kappa5_2 = np.array([])
roc5_2 = np.array([])
fpr5_2 = np.array([])

precision5_3 = np.array([])
recall5_3 = np.array([])
f1_score5_3 = np.array([])
kappa5_3 = np.array([])
roc5_3 = np.array([])
fpr5_3 = np.array([])

precision5_4 = np.array([])
recall5_4 = np.array([])
f1_score5_4 = np.array([])
kappa5_4 = np.array([])
roc5_4 = np.array([])
fpr5_4 = np.array([])


precision6_1 = np.array([])
recall6_1 = np.array([])
f1_score6_1 = np.array([])
kappa6_1 = np.array([])
roc6_1 = np.array([])
fpr6_1 = np.array([])

precision6_2 = np.array([])
recall6_2 = np.array([])
f1_score6_2 = np.array([])
kappa6_2 = np.array([])
roc6_2 = np.array([])
fpr6_2 = np.array([])

precision6_3 = np.array([])
recall6_3 = np.array([])
f1_score6_3 = np.array([])
kappa6_3 = np.array([])
roc6_3 = np.array([])
fpr6_3 = np.array([])

precision6_4 = np.array([])
recall6_4 = np.array([])
f1_score6_4 = np.array([])
kappa6_4 = np.array([])
roc6_4 = np.array([])
fpr6_4 = np.array([])


precision7_1 = np.array([])
recall7_1 = np.array([])
f1_score7_1 = np.array([])
kappa7_1 = np.array([])
roc7_1 = np.array([])
fpr7_1 = np.array([])

precision7_2 = np.array([])
recall7_2 = np.array([])
f1_score7_2 = np.array([])
kappa7_2 = np.array([])
roc7_2 = np.array([])
fpr7_2 = np.array([])

precision7_3 = np.array([])
recall7_3 = np.array([])
f1_score7_3 = np.array([])
kappa7_3 = np.array([])
roc7_3 = np.array([])
fpr7_3 = np.array([])

precision7_4 = np.array([])
recall7_4 = np.array([])
f1_score7_4 = np.array([])
kappa7_4 = np.array([])
roc7_4 = np.array([])
fpr7_4 = np.array([])


precision8_1 = np.array([])
recall8_1 = np.array([])
f1_score8_1 = np.array([])
kappa8_1 = np.array([])
roc8_1 = np.array([])
fpr8_1 = np.array([])

precision8_2 = np.array([])
recall8_2 = np.array([])
f1_score8_2 = np.array([])
kappa8_2 = np.array([])
roc8_2 = np.array([])
fpr8_2 = np.array([])

precision8_3 = np.array([])
recall8_3 = np.array([])
f1_score8_3 = np.array([])
kappa8_3 = np.array([])
roc8_3 = np.array([])
fpr8_3 = np.array([])

precision8_4 = np.array([])
recall8_4 = np.array([])
f1_score8_4 = np.array([])
kappa8_4 = np.array([])
roc8_4 = np.array([])
fpr8_4 = np.array([])


precision_s4 = np.array([])
recall_s4 = np.array([])
f1_s4 = np.array([])
kappa_s4 = np.array([])
roc_s4 = np.array([])
fpr_s4 = np.array([])

precision_s3 = np.array([])
recall_s3 = np.array([])
f1_s3 = np.array([])
kappa_s3 = np.array([])
roc_s3 = np.array([])
fpr_s3 = np.array([])

precision_s2 = np.array([])
recall_s2 = np.array([])
f1_s2 = np.array([])
kappa_s2 = np.array([])
roc_s2 = np.array([])
fpr_s2 = np.array([])

precision_s5 = np.array([])
recall_s5 = np.array([])
f1_s5 = np.array([])
kappa_s5 = np.array([])
roc_s5 = np.array([])
fpr_s5 = np.array([])

precision_s6 = np.array([])
recall_s6 = np.array([])
f1_s6 = np.array([])
kappa_s6 = np.array([])
roc_s6 = np.array([])
fpr_s6 = np.array([])

precision_s7 = np.array([])
recall_s7 = np.array([])
f1_s7 = np.array([])
kappa_s7 = np.array([])
roc_s7 = np.array([])
fpr_s7 = np.array([])

precision_s8 = np.array([])
recall_s8 = np.array([])
f1_s8 = np.array([])
kappa_s8 = np.array([])
roc_s8 = np.array([])
fpr_s8 = np.array([])

In [23]:
########## different feature sets #############
col = []    ### all ######
col1 = []   ### iraw_oef ######
col2 = []   ### idur_oef ######

for i in range(0,347):
  col = np.append(col, i)

for m in range(0,177):
  col2 = np.append(col2, m)

for n in range(0,7):
  col1 = np.append(col1, n)
for l in range(177,347):
  col1 = np.append(col1, l)

def feature_all(X):
    return X

def feature_1(X):
    return X[ : , col1.astype(int)]
  
def feature_2(X):
    return X[ : , col2.astype(int)]



In [24]:
# Import Libraries
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,Y)
columns = X.columns.values

print(model.feature_importances_)
feat_importances = pd.Series(model.feature_importances_)

[0.00305992 0.01715327 0.02569513 0.0129098  0.00766266 0.0061039
 0.00515771 0.00312132 0.00294128 0.00366641 0.00280578 0.00159336
 0.00416958 0.00388117 0.00262705 0.0034953  0.00392244 0.00416049
 0.00280925 0.00526893 0.00389785 0.00387259 0.00331333 0.00345672
 0.00424721 0.00395622 0.00145385 0.00320479 0.00137773 0.00400222
 0.00275688 0.00319841 0.00440288 0.00388969 0.00274712 0.00312786
 0.00364526 0.00247336 0.00228824 0.00258284 0.00412645 0.00444093
 0.00453543 0.00312705 0.00411878 0.00482706 0.0042392  0.00419175
 0.00308294 0.0017433  0.00340325 0.00258663 0.00344961 0.00256004
 0.00619367 0.00526051 0.00223033 0.00256371 0.00230891 0.00244086
 0.00452813 0.00350741 0.0054328  0.00324911 0.00234745 0.00353349
 0.00455738 0.00188639 0.00284743 0.00239259 0.00226252 0.00394498
 0.00319236 0.00458148 0.00430266 0.00351618 0.00425396 0.00485292
 0.00356066 0.00117198 0.00364046 0.00250896 0.00257888 0.00513911
 0.0050175  0.00322929 0.00252093 0.00393749 0.00296628 0.00327

In [25]:
# Print 10 Best Features

top = feat_importances.nlargest(10)
feature = top.index
names = columns[feature]

top_features = pd.DataFrame()
top_features["Features"]=names
top_features["    Score"]=top.values
top_features["Ranking"] = ["       --->      Rank-01","       --->      Rank-02","       --->      Rank-03","       --->      Rank-04","       --->      Rank-05","       --->      Rank-06","       --->      Rank-07","       --->      Rank-08","       --->      Rank-09","       --->      Rank-10"]
print("\n===================================================\n")
print("================== TOP 10 FEATURES ================")
print("\n===================================================\n")
print(top_features.to_string(index=False))
print("\n===================================================\n")



================== TOP 10 FEATURES ================


   Features      Score                  Ranking
  mean_idur   0.025695        --->      Rank-01
   tot_time   0.017153        --->      Rank-02
median_idur   0.012910        --->      Rank-03
   idur.146   0.009350        --->      Rank-04
   idur.142   0.008493        --->      Rank-05
    idur.93   0.007971        --->      Rank-06
   max_idur   0.007663        --->      Rank-07
   iraw.157   0.007587        --->      Rank-08
    idur.84   0.006868        --->      Rank-09
    idur.48   0.006194        --->      Rank-10




In [26]:
Features_Set2 = X[top_features["Features"]]

print("\n\nAutomatic Selected Features Set 2:")
print("==================================\n")
Features_Set2



Automatic Selected Features Set 2:



,mean_idur,tot_time,median_idur,idur.146,idur.142,idur.93,max_idur,iraw.157,idur.84,idur.48
0,0.254294,0.251140,0.246154,0.125261,0.200000,0.087379,0.037668,0.0,0.063361,0.088608
1,0.379208,0.373250,0.379487,0.060543,0.126984,0.031553,0.090583,0.0,0.071625,0.071730
2,0.382986,0.375825,0.405128,0.198330,0.187302,0.050971,0.069955,1.0,0.179063,0.236287
3,0.318125,0.318526,0.369231,0.156576,0.082540,0.058252,0.047534,1.0,0.085399,0.210970
4,0.451111,0.440850,0.441026,0.185804,0.088889,0.024272,0.078924,1.0,0.107438,0.092827
...,...,...,...,...,...,...,...,...,...,...
1631,0.179700,0.177477,0.174359,0.075157,0.066667,0.033981,0.045740,1.0,0.132231,0.075949
1632,0.283604,0.274478,0.235897,0.114823,0.047619,0.075243,0.115695,1.0,0.046832,0.223629
1633,0.386364,0.375396,0.215385,0.221294,0.123810,0.067961,0.198206,1.0,0.041322,0.358650
1634,0.147756,0.147916,0.082051,0.068894,0.031746,0.067961,0.140807,1.0,0.035813,0.278481


In [27]:

i = 0
for train_ix, test_ix in k_fold.split(Features_Set2):
    i = i + 1
    sheet = wb.add_sheet('fold'+str(i), cell_overwrite_ok=True) 

    sheet.write(0, 0, 'Model')
    sheet.write(0, 1, 'dataset')
    sheet.write(0, 2, 'threshold')
    sheet.write(0, 3, 'Precision')
    sheet.write(0, 4, 'Recall')
    sheet.write(0, 5, 'F1 Score')
    sheet.write(0, 6, 'weighted Kappa')
    sheet.write(0, 7, 'ROC AUC')
    sheet.write(0, 8, 'FPR')
    sheet.write(0, 9, 'tp')
    sheet.write(0, 10, 'fn')
    sheet.write(0, 11, 'fp')
    sheet.write(0, 12, 'tn')
    sheet.write(0, 13, 'total test samples')
    sheet.write(0, 14, 'rate of predicted cheaters')
    sheet.write(0, 15, 'real cheaters in testset')
    sheet.write(0, 16, 'total train samples')

    # kfold
    # train_x, train_y, test_x, test_y = Features_Set2[train_ix], Y[train_ix], Features_Set2[test_ix], Y[test_ix]
    train_x, train_y, test_x, test_y = Features_Set2.iloc[train_ix], Y[train_ix], Features_Set2.iloc[test_ix], Y[test_ix]


    ###### SMOTE and RandomUnderSampler (for balancing data) ############

    # both 33%
    # over = SMOTE(sampling_strategy=0.4)
    # under = RandomUnderSampler(sampling_strategy=0.5)
    # steps = [('o', over), ('u', under)]

    # both 50%
    over = SMOTE(sampling_strategy=0.4, random_state=20)
    under = RandomUnderSampler(sampling_strategy=1, random_state=20)
    steps = [('o', over), ('u', under)]

    # only smote 50%
    # over = SMOTE(sampling_strategy=1)
    # steps = [('o', over)]

    # # only smote 33%
    # over = SMOTE(sampling_strategy=0.5)
    # steps = [('o', over)]

    # # only undersampling 50%
    # under = RandomUnderSampler(sampling_strategy=1)
    # steps = [('u', under)]

    # # only undersampling 33%
    # under = RandomUnderSampler(sampling_strategy=0.5)
    # steps = [('u', under)]
    
    # both 50% with high over
    # over = SMOTE(sampling_strategy=0.8)
    # under = RandomUnderSampler(sampling_strategy=1)
    # steps = [('o', over), ('u', under)]

    pipeline = Pipeline(steps=steps)

    x_smote, y_smote = pipeline.fit_resample(train_x, train_y)

    # x1_smote, y1_smote = pipeline1.fit_resample(train_x1, train_y)
    # x2_smote, y2_smote = pipeline2.fit_resample(train_x2, train_y)

    #### no resampling ###
    # x_smote, y_smote = train_x, train_y

    size_trainset = len(y_smote)

 ###### eight models #################
    ###
#     model1 = DecisionTreeClassifier(criterion="entropy", max_features="auto", splitter="best")
#     # Fit, Predict
#     model1.fit(x_smote, y_smote)
#     probs1 = model1.predict_proba(test_x)
#     probs1 = probs1[:, 1]

#     pred1 = (probs1 >= 0.7).astype(bool) # set threshold as 0.7

#     tn, fp, fn, tp = confusion_matrix(test_y, pred1).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs1)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(1, 0, "Decision Tree")
#     sheet.write(1, 1, "iraw_idur_OEF")
#     sheet.write(1, 2, "0.7")
#     sheet.write(1, 3, precision_score(test_y, pred1))
#     sheet.write(1, 4, recall_score(test_y, pred1))
#     sheet.write(1, 5, f1_score(test_y, pred1))
#     sheet.write(1, 6, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
#     sheet.write(1, 7, auc)
#     sheet.write(1, 8, fpr)

#     sheet.write(1, 9, str(tp))
#     sheet.write(1, 10, str(fn))
#     sheet.write(1, 11, str(fp))
#     sheet.write(1, 12, str(tn))
#     sheet.write(1, 13, str(tot_n))
#     sheet.write(1, 14, str(fptp_rate))
#     sheet.write(1, 15, str(real_cheaters))
#     sheet.write(1, 16, str(size_trainset))

#     precision1_1 = np.append(precision1_1, precision_score(test_y, pred1))
#     recall1_1 = np.append(recall1_1, recall_score(test_y, pred1))
#     f1_score1_1 = np.append(f1_score1_1, f1_score(test_y, pred1))
#     kappa1_1 = np.append(kappa1_1, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
#     roc1_1 = np.append(roc1_1, auc)
#     fpr1_1 = np.append(fpr1_1, fpr)


#     ###########
#     pred1 = (probs1 >= 0.5).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred1).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs1)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(2, 0, "Decision Tree")
#     sheet.write(2, 1, "iraw_idur_OEF")
#     sheet.write(2, 2, "0.5")
#     sheet.write(2, 3, precision_score(test_y, pred1))
#     sheet.write(2, 4, recall_score(test_y, pred1))
#     sheet.write(2, 5, f1_score(test_y, pred1))
#     sheet.write(2, 6, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
#     sheet.write(2, 7, auc)
#     sheet.write(2, 8, fpr)

#     sheet.write(2, 9, str(tp))
#     sheet.write(2, 10, str(fn))
#     sheet.write(2, 11, str(fp))
#     sheet.write(2, 12, str(tn))
#     sheet.write(2, 13, str(tot_n))
#     sheet.write(2, 14, str(fptp_rate))
#     sheet.write(2, 15, str(real_cheaters))
#     sheet.write(2, 16, str(size_trainset))

#     precision1_2 = np.append(precision1_2, precision_score(test_y, pred1))
#     recall1_2 = np.append(recall1_2, recall_score(test_y, pred1))
#     f1_score1_2 = np.append(f1_score1_2, f1_score(test_y, pred1))
#     kappa1_2 = np.append(kappa1_2, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
#     roc1_2 = np.append(roc1_2, auc)
#     fpr1_2 = np.append(fpr1_2, fpr)

#     ###########
#     pred1 = (probs1 >= 0.97).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred1).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs1)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(3, 0, "Decision Tree")
#     sheet.write(3, 1, "iraw_idur_OEF")
#     sheet.write(3, 2, "0.97")
#     sheet.write(3, 3, precision_score(test_y, pred1))
#     sheet.write(3, 4, recall_score(test_y, pred1))
#     sheet.write(3, 5, f1_score(test_y, pred1))
#     sheet.write(3, 6, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
#     sheet.write(3, 7, auc)
#     sheet.write(3, 8, fpr)

#     sheet.write(3, 9, str(tp))
#     sheet.write(3, 10, str(fn))
#     sheet.write(3, 11, str(fp))
#     sheet.write(3, 12, str(tn))
#     sheet.write(3, 13, str(tot_n))
#     sheet.write(3, 14, str(fptp_rate))
#     sheet.write(3, 15, str(real_cheaters))
#     sheet.write(3, 16, str(size_trainset))

#     precision1_3 = np.append(precision1_3, precision_score(test_y, pred1))
#     recall1_3 = np.append(recall1_3, recall_score(test_y, pred1))
#     f1_score1_3 = np.append(f1_score1_3, f1_score(test_y, pred1))
#     kappa1_3 = np.append(kappa1_3, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
#     roc1_3 = np.append(roc1_3, auc)
#     fpr1_3 = np.append(fpr1_3, fpr)



# ############################    ###
#     model2 = SVC(kernel='linear', probability=True)
#     # Fit, Predict
#     model2.fit(x_smote, y_smote)
#     probs2 = model2.predict_proba(test_x)
#     probs2 = probs2[:, 1]

#     pred2 = (probs2 >= 0.7).astype(bool) # set threshold as 0.7

#     tn, fp, fn, tp = confusion_matrix(test_y, pred2).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs2)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(4, 0, "Support Vector Machine")
#     sheet.write(4, 1, "iraw_idur_OEF")
#     sheet.write(4, 2, "0.7")
#     sheet.write(4, 3, precision_score(test_y, pred2))
#     sheet.write(4, 4, recall_score(test_y, pred2))
#     sheet.write(4, 5, f1_score(test_y, pred2))
#     sheet.write(4, 6, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
#     sheet.write(4, 7, auc)
#     sheet.write(4, 8, fpr)

#     sheet.write(4, 9, str(tp))
#     sheet.write(4, 10, str(fn))
#     sheet.write(4, 11, str(fp))
#     sheet.write(4, 12, str(tn))
#     sheet.write(4, 13, str(tot_n))
#     sheet.write(4, 14, str(fptp_rate))
#     sheet.write(4, 15, str(real_cheaters))
#     sheet.write(4, 16, str(size_trainset))

#     precision2_1 = np.append(precision2_1, precision_score(test_y, pred2))
#     recall2_1 = np.append(recall2_1, recall_score(test_y, pred2))
#     f1_score2_1 = np.append(f1_score2_1, f1_score(test_y, pred2))
#     kappa2_1 = np.append(kappa2_1, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
#     roc2_1 = np.append(roc2_1, auc)
#     fpr2_1 = np.append(fpr2_1, fpr)


#     ############
#     pred2 = (probs2 >= 0.5).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred2).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs2)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(5, 0, "Support Vector Machine")
#     sheet.write(5, 1, "iraw_idur_OEF")
#     sheet.write(5, 2, "0.5")
#     sheet.write(5, 3, precision_score(test_y, pred2))
#     sheet.write(5, 4, recall_score(test_y, pred2))
#     sheet.write(5, 5, f1_score(test_y, pred2))
#     sheet.write(5, 6, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
#     sheet.write(5, 7, auc)
#     sheet.write(5, 8, fpr)
#     sheet.write(5, 9, str(tp))
#     sheet.write(5, 10, str(fn))
#     sheet.write(5, 11, str(fp))
#     sheet.write(5, 12, str(tn))
#     sheet.write(5, 13, str(tot_n))
#     sheet.write(5, 14, str(fptp_rate))
#     sheet.write(5, 15, str(real_cheaters))
#     sheet.write(5, 16, str(size_trainset))

#     precision2_2 = np.append(precision2_2, precision_score(test_y, pred2))
#     recall2_2 = np.append(recall2_2, recall_score(test_y, pred2))
#     f1_score2_2 = np.append(f1_score2_2, f1_score(test_y, pred2))
#     kappa2_2 = np.append(kappa2_2, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
#     roc2_2 = np.append(roc2_2, auc)
#     fpr2_2 = np.append(fpr2_2, fpr)

#    ############
#     pred2 = (probs2 >= 0.97).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred2).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs2)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(6, 0, "Support Vector Machine")
#     sheet.write(6, 1, "iraw_idur_OEF")
#     sheet.write(6, 2, "0.97")
#     sheet.write(6, 3, precision_score(test_y, pred2))
#     sheet.write(6, 4, recall_score(test_y, pred2))
#     sheet.write(6, 5, f1_score(test_y, pred2))
#     sheet.write(6, 6, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
#     sheet.write(6, 7, auc)
#     sheet.write(6, 8, fpr)
#     sheet.write(6, 9, str(tp))
#     sheet.write(6, 10, str(fn))
#     sheet.write(6, 11, str(fp))
#     sheet.write(6, 12, str(tn))
#     sheet.write(6, 13, str(tot_n))
#     sheet.write(6, 14, str(fptp_rate))
#     sheet.write(6, 15, str(real_cheaters))
#     sheet.write(6, 16, str(size_trainset))

#     precision2_3 = np.append(precision2_3, precision_score(test_y, pred2))
#     recall2_3 = np.append(recall2_3, recall_score(test_y, pred2))
#     f1_score2_3 = np.append(f1_score2_3, f1_score(test_y, pred2))
#     kappa2_3 = np.append(kappa2_3, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
#     roc2_3 = np.append(roc2_3, auc)
#     fpr2_3 = np.append(fpr2_3, fpr)



# ###################################
#     ###
#     model3 = LogisticRegression(C=2, multi_class="multinomial", solver="newton-cg")
#     # Fit, Predict
#     model3.fit(x_smote, y_smote)
#     probs3 = model3.predict_proba(test_x)
#     probs3 = probs3[:, 1]

#     pred3 = (probs3 >= 0.7).astype(bool) # set threshold as 0.7

#     tn, fp, fn, tp = confusion_matrix(test_y, pred3).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs3)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(7, 0, "Logistic Regression")
#     sheet.write(7, 1, "iraw_idur_OEF")
#     sheet.write(7, 2, "0.7")
#     sheet.write(7, 3, precision_score(test_y, pred3))
#     sheet.write(7, 4, recall_score(test_y, pred3))
#     sheet.write(7, 5, f1_score(test_y, pred3))
#     sheet.write(7, 6, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
#     sheet.write(7, 7, auc)
#     sheet.write(7, 8, fpr)
#     sheet.write(7, 9, str(tp))
#     sheet.write(7, 10, str(fn))
#     sheet.write(7, 11, str(fp))
#     sheet.write(7, 12, str(tn))
#     sheet.write(7, 13, str(tot_n))
#     sheet.write(7, 14, str(fptp_rate))
#     sheet.write(7, 15, str(real_cheaters))
#     sheet.write(7, 16, str(size_trainset))

#     precision3_1 = np.append(precision3_1, precision_score(test_y, pred3))
#     recall3_1 = np.append(recall3_1, recall_score(test_y, pred3))
#     f1_score3_1 = np.append(f1_score3_1, f1_score(test_y, pred3))
#     kappa3_1 = np.append(kappa3_1, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
#     roc3_1 = np.append(roc3_1, auc)
#     fpr3_1 = np.append(fpr3_1, fpr)

#     ######
#     pred3 = (probs3 >= 0.5).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred3).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs3)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(8, 0, "Logistic Regression")
#     sheet.write(8, 1, "iraw_idur_OEF")
#     sheet.write(8, 2, "0.5")
#     sheet.write(8, 3, precision_score(test_y, pred3))
#     sheet.write(8, 4, recall_score(test_y, pred3))
#     sheet.write(8, 5, f1_score(test_y, pred3))
#     sheet.write(8, 6, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
#     sheet.write(8, 7, auc)
#     sheet.write(8, 8, fpr)
#     sheet.write(8, 9, str(tp))
#     sheet.write(8, 10, str(fn))
#     sheet.write(8, 11, str(fp))
#     sheet.write(8, 12, str(tn))
#     sheet.write(8, 13, str(tot_n))
#     sheet.write(8, 14, str(fptp_rate))
#     sheet.write(8, 15, str(real_cheaters))
#     sheet.write(8, 16, str(size_trainset))

#     precision3_2 = np.append(precision3_2, precision_score(test_y, pred3))
#     recall3_2 = np.append(recall3_2, recall_score(test_y, pred3))
#     f1_score3_2 = np.append(f1_score3_2, f1_score(test_y, pred3))
#     kappa3_2 = np.append(kappa3_2, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
#     roc3_2 = np.append(roc3_2, auc)
#     fpr3_2 = np.append(fpr3_2, fpr)

#     ######
#     pred3 = (probs3 >= 0.97).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred3).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs3)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(9, 0, "Logistic Regression")
#     sheet.write(9, 1, "iraw_idur_OEF")
#     sheet.write(9, 2, "0.97")
#     sheet.write(9, 3, precision_score(test_y, pred3))
#     sheet.write(9, 4, recall_score(test_y, pred3))
#     sheet.write(9, 5, f1_score(test_y, pred3))
#     sheet.write(9, 6, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
#     sheet.write(9, 7, auc)
#     sheet.write(9, 8, fpr)
#     sheet.write(9, 9, str(tp))
#     sheet.write(9, 10, str(fn))
#     sheet.write(9, 11, str(fp))
#     sheet.write(9, 12, str(tn))
#     sheet.write(9, 13, str(tot_n))
#     sheet.write(9, 14, str(fptp_rate))
#     sheet.write(9, 15, str(real_cheaters))
#     sheet.write(9, 16, str(size_trainset))

#     precision3_3 = np.append(precision3_3, precision_score(test_y, pred3))
#     recall3_3 = np.append(recall3_3, recall_score(test_y, pred3))
#     f1_score3_3 = np.append(f1_score3_3, f1_score(test_y, pred3))
#     kappa3_3 = np.append(kappa3_3, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
#     roc3_3 = np.append(roc3_3, auc)
#     fpr3_3 = np.append(fpr3_3, fpr)

 

# #########################################
#     ###
#     model4 = LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen")
#     # Fit, Predict
#     model4.fit(x_smote, y_smote)
#     probs4 = model4.predict_proba(test_x)
#     probs4 = probs4[:, 1]

#     pred4 = (probs4 >= 0.7).astype(bool) # set threshold as 0.7

#     tn, fp, fn, tp = confusion_matrix(test_y, pred4).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs4)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(10, 0, "Discriminant Analysis")
#     sheet.write(10, 1, "iraw_idur_OEF")
#     sheet.write(10, 2, "0.7")
#     sheet.write(10, 3, precision_score(test_y, pred4))
#     sheet.write(10, 4, recall_score(test_y, pred4))
#     sheet.write(10, 5, f1_score(test_y, pred4))
#     sheet.write(10, 6, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
#     sheet.write(10, 7, auc)
#     sheet.write(10, 8, fpr)
#     sheet.write(10, 9, str(tp))
#     sheet.write(10, 10, str(fn))
#     sheet.write(10, 11, str(fp))
#     sheet.write(10, 12, str(tn))
#     sheet.write(10, 13, str(tot_n))
#     sheet.write(10, 14, str(fptp_rate))
#     sheet.write(10, 15, str(real_cheaters))
#     sheet.write(10, 16, str(size_trainset))

#     precision4_1 = np.append(precision4_1, precision_score(test_y, pred4))
#     recall4_1 = np.append(recall4_1, recall_score(test_y, pred4))
#     f1_score4_1 = np.append(f1_score4_1, f1_score(test_y, pred4))
#     kappa4_1 = np.append(kappa4_1, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
#     roc4_1 = np.append(roc4_1, auc)
#     fpr4_1 = np.append(fpr4_1, fpr)

#     ############
#     pred4 = (probs4 >= 0.5).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred4).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs4)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(11, 0, "Discriminant Analysis")
#     sheet.write(11, 1, "iraw_idur_OEF")
#     sheet.write(11, 2, "0.5")
#     sheet.write(11, 3, precision_score(test_y, pred4))
#     sheet.write(11, 4, recall_score(test_y, pred4))
#     sheet.write(11, 5, f1_score(test_y, pred4))
#     sheet.write(11, 6, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
#     sheet.write(11, 7, auc)
#     sheet.write(11, 8, fpr)
#     sheet.write(11, 9, str(tp))
#     sheet.write(11, 10, str(fn))
#     sheet.write(11, 11, str(fp))
#     sheet.write(11, 12, str(tn))
#     sheet.write(11, 13, str(tot_n))
#     sheet.write(11, 14, str(fptp_rate))
#     sheet.write(11, 15, str(real_cheaters))
#     sheet.write(11, 16, str(size_trainset))

#     precision4_2 = np.append(precision4_2, precision_score(test_y, pred4))
#     recall4_2 = np.append(recall4_2, recall_score(test_y, pred4))
#     f1_score4_2 = np.append(f1_score4_2, f1_score(test_y, pred4))
#     kappa4_2 = np.append(kappa4_2, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
#     roc4_2 = np.append(roc4_2, auc)
#     fpr4_2 = np.append(fpr4_2, fpr)

#     ############
#     pred4 = (probs4 >= 0.97).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred4).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs4)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(12, 0, "Discriminant Analysis")
#     sheet.write(12, 1, "iraw_idur_OEF")
#     sheet.write(12, 2, "0.97")
#     sheet.write(12, 3, precision_score(test_y, pred4))
#     sheet.write(12, 4, recall_score(test_y, pred4))
#     sheet.write(12, 5, f1_score(test_y, pred4))
#     sheet.write(12, 6, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
#     sheet.write(12, 7, auc)
#     sheet.write(12, 8, fpr)
#     sheet.write(12, 9, str(tp))
#     sheet.write(12, 10, str(fn))
#     sheet.write(12, 11, str(fp))
#     sheet.write(12, 12, str(tn))
#     sheet.write(12, 13, str(tot_n))
#     sheet.write(12, 14, str(fptp_rate))
#     sheet.write(12, 15, str(real_cheaters))
#     sheet.write(12, 16, str(size_trainset))

#     precision4_3 = np.append(precision4_3, precision_score(test_y, pred4))
#     recall4_3 = np.append(recall4_3, recall_score(test_y, pred4))
#     f1_score4_3 = np.append(f1_score4_3, f1_score(test_y, pred4))
#     kappa4_3 = np.append(kappa4_3, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
#     roc4_3 = np.append(roc4_3, auc)
#     fpr4_3 = np.append(fpr4_3, fpr)

 
 ###############################################
#     ###
#     model5 = GaussianNB(var_smoothing=6.579332246575682e-08)
#     # Fit, Predict
#     model5.fit(x_smote, y_smote)
#     probs5 = model5.predict_proba(test_x)
#     probs5 = probs5[:, 1]

#     pred5 = (probs5 >= 0.7).astype(bool) # set threshold as 0.7

#     tn, fp, fn, tp = confusion_matrix(test_y, pred5).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs5)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(13, 0, "Naive Bayes")
#     sheet.write(13, 1, "iraw_idur_OEF")
#     sheet.write(13, 2, "0.7")
#     sheet.write(13, 3, precision_score(test_y, pred5))
#     sheet.write(13, 4, recall_score(test_y, pred5))
#     sheet.write(13, 5, f1_score(test_y, pred5))
#     sheet.write(13, 6, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
#     sheet.write(13, 7, auc)
#     sheet.write(13, 8, fpr)
#     sheet.write(13, 9, str(tp))
#     sheet.write(13, 10, str(fn))
#     sheet.write(13, 11, str(fp))
#     sheet.write(13, 12, str(tn))
#     sheet.write(13, 13, str(tot_n))
#     sheet.write(13, 14, str(fptp_rate))
#     sheet.write(13, 15, str(real_cheaters))
#     sheet.write(13, 16, str(size_trainset))

#     precision5_1 = np.append(precision5_1, precision_score(test_y, pred5))
#     recall5_1 = np.append(recall5_1, recall_score(test_y, pred5))
#     f1_score5_1 = np.append(f1_score5_1, f1_score(test_y, pred5))
#     kappa5_1 = np.append(kappa5_1, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
#     roc5_1 = np.append(roc5_1, auc)
#     fpr5_1 = np.append(fpr5_1, fpr)


#     ###############
#     pred5 = (probs5 >= 0.5).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred5).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs5)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(14, 0, "Naive Bayes")
#     sheet.write(14, 1, "iraw_idur_OEF")
#     sheet.write(14, 2, "0.5")
#     sheet.write(14, 3, precision_score(test_y, pred5))
#     sheet.write(14, 4, recall_score(test_y, pred5))
#     sheet.write(14, 5, f1_score(test_y, pred5))
#     sheet.write(14, 6, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
#     sheet.write(14, 7, auc)
#     sheet.write(14, 8, fpr)
#     sheet.write(14, 9, str(tp))
#     sheet.write(14, 10, str(fn))
#     sheet.write(14, 11, str(fp))
#     sheet.write(14, 12, str(tn))
#     sheet.write(14, 13, str(tot_n))
#     sheet.write(14, 14, str(fptp_rate))
#     sheet.write(14, 15, str(real_cheaters))
#     sheet.write(14, 16, str(size_trainset))

#     precision5_2 = np.append(precision5_2, precision_score(test_y, pred5))
#     recall5_2 = np.append(recall5_2, recall_score(test_y, pred5))
#     f1_score5_2 = np.append(f1_score5_2, f1_score(test_y, pred5))
#     kappa5_2 = np.append(kappa5_2, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
#     roc5_2 = np.append(roc5_2, auc)
#     fpr5_2 = np.append(fpr5_2, fpr)

#     ###############
#     pred5 = (probs5 >= 0.97).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred5).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs5)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(15, 0, "Naive Bayes")
#     sheet.write(15, 1, "iraw_idur_OEF")
#     sheet.write(15, 2, "0.97")
#     sheet.write(15, 3, precision_score(test_y, pred5))
#     sheet.write(15, 4, recall_score(test_y, pred5))
#     sheet.write(15, 5, f1_score(test_y, pred5))
#     sheet.write(15, 6, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
#     sheet.write(15, 7, auc)
#     sheet.write(15, 8, fpr)
#     sheet.write(15, 9, str(tp))
#     sheet.write(15, 10, str(fn))
#     sheet.write(15, 11, str(fp))
#     sheet.write(15, 12, str(tn))
#     sheet.write(15, 13, str(tot_n))
#     sheet.write(15, 14, str(fptp_rate))
#     sheet.write(15, 15, str(real_cheaters))
#     sheet.write(15, 16, str(size_trainset))

#     precision5_3 = np.append(precision5_3, precision_score(test_y, pred5))
#     recall5_3 = np.append(recall5_3, recall_score(test_y, pred5))
#     f1_score5_3 = np.append(f1_score5_3, f1_score(test_y, pred5))
#     kappa5_3 = np.append(kappa5_3, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
#     roc5_3 = np.append(roc5_3, auc)
#     fpr5_3 = np.append(fpr5_3, fpr)

 
# ##############################################################
#     ###
#     model6 = MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam")
#     # Fit, Predict
#     model6.fit(x_smote, y_smote)
#     probs6 = model6.predict_proba(test_x)
#     probs6 = probs6[:, 1]

#     pred6 = (probs6 >= 0.7).astype(bool) # set threshold as 0.7

#     tn, fp, fn, tp = confusion_matrix(test_y, pred6).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs6)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(16, 0, "Neural Network")
#     sheet.write(16, 1, "iraw_idur_OEF")
#     sheet.write(16, 2, "0.7")
#     sheet.write(16, 3, precision_score(test_y, pred6))
#     sheet.write(16, 4, recall_score(test_y, pred6))
#     sheet.write(16, 5, f1_score(test_y, pred6))
#     sheet.write(16, 6, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
#     sheet.write(16, 7, auc)
#     sheet.write(16, 8, fpr)
#     sheet.write(16, 9, str(tp))
#     sheet.write(16, 10, str(fn))
#     sheet.write(16, 11, str(fp))
#     sheet.write(16, 12, str(tn))
#     sheet.write(16, 13, str(tot_n))
#     sheet.write(16, 14, str(fptp_rate))
#     sheet.write(16, 15, str(real_cheaters))
#     sheet.write(16, 16, str(size_trainset))

#     precision6_1 = np.append(precision6_1, precision_score(test_y, pred6))
#     recall6_1 = np.append(recall6_1, recall_score(test_y, pred6))
#     f1_score6_1 = np.append(f1_score6_1, f1_score(test_y, pred6))
#     kappa6_1 = np.append(kappa6_1, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
#     roc6_1 = np.append(roc6_1, auc)
#     fpr6_1 = np.append(fpr6_1, fpr)

#     ##################
#     pred6 = (probs6 >= 0.5).astype(bool)

#     tn, fp, fn, tp = confusion_matrix(test_y, pred6).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs6)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(17, 0, "Neural Network")
#     sheet.write(17, 1, "iraw_idur_OEF")
#     sheet.write(17, 2, "0.5")
#     sheet.write(17, 3, precision_score(test_y, pred6))
#     sheet.write(17, 4, recall_score(test_y, pred6))
#     sheet.write(17, 5, f1_score(test_y, pred6))
#     sheet.write(17, 6, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
#     sheet.write(17, 7, auc)
#     sheet.write(17, 8, fpr)
#     sheet.write(17, 9, str(tp))
#     sheet.write(17, 10, str(fn))
#     sheet.write(17, 11, str(fp))
#     sheet.write(17, 12, str(tn))
#     sheet.write(17, 13, str(tot_n))
#     sheet.write(17, 14, str(fptp_rate))
#     sheet.write(17, 15, str(real_cheaters))
#     sheet.write(17, 16, str(size_trainset))

#     precision6_2 = np.append(precision6_2, precision_score(test_y, pred6))
#     recall6_2 = np.append(recall6_2, recall_score(test_y, pred6))
#     f1_score6_2 = np.append(f1_score6_2, f1_score(test_y, pred6))
#     kappa6_2 = np.append(kappa6_2, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
#     roc6_2 = np.append(roc6_2, auc)
#     fpr6_2 = np.append(fpr6_2, fpr)

#     ##################
#     pred6 = (probs6 >= 0.97).astype(bool)

#     tn, fp, fn, tp = confusion_matrix(test_y, pred6).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs6)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(18, 0, "Neural Network")
#     sheet.write(18, 1, "iraw_idur_OEF")
#     sheet.write(18, 2, "0.97")
#     sheet.write(18, 3, precision_score(test_y, pred6))
#     sheet.write(18, 4, recall_score(test_y, pred6))
#     sheet.write(18, 5, f1_score(test_y, pred6))
#     sheet.write(18, 6, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
#     sheet.write(18, 7, auc)
#     sheet.write(18, 8, fpr)
#     sheet.write(18, 9, str(tp))
#     sheet.write(18, 10, str(fn))
#     sheet.write(18, 11, str(fp))
#     sheet.write(18, 12, str(tn))
#     sheet.write(18, 13, str(tot_n))
#     sheet.write(18, 14, str(fptp_rate))
#     sheet.write(18, 15, str(real_cheaters))
#     sheet.write(18, 16, str(size_trainset))

#     precision6_3 = np.append(precision6_3, precision_score(test_y, pred6))
#     recall6_3 = np.append(recall6_3, recall_score(test_y, pred6))
#     f1_score6_3 = np.append(f1_score6_3, f1_score(test_y, pred6))
#     kappa6_3 = np.append(kappa6_3, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
#     roc6_3 = np.append(roc6_3, auc)
#     fpr6_3 = np.append(fpr6_3, fpr)

 
# ############################################
#     ###
#     model7 = RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
#     # Fit, Predict
#     model7.fit(x_smote, y_smote)
#     probs7 = model7.predict_proba(test_x)
#     probs7 = probs7[:, 1]

#     pred7 = (probs7 >= 0.7).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred7).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs7)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(19, 0, "Random Forest")
#     sheet.write(19, 1, "idur_OEF")
#     sheet.write(19, 2, "0.7")
#     sheet.write(19, 3, precision_score(test_y, pred7))
#     sheet.write(19, 4, recall_score(test_y, pred7))
#     sheet.write(19, 5, f1_score(test_y, pred7))
#     sheet.write(19, 6, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
#     sheet.write(19, 7, auc)
#     sheet.write(19, 8, fpr)
#     sheet.write(19, 9, str(tp))
#     sheet.write(19, 10, str(fn))
#     sheet.write(19, 11, str(fp))
#     sheet.write(19, 12, str(tn))
#     sheet.write(19, 13, str(tot_n))
#     sheet.write(19, 14, str(fptp_rate))
#     sheet.write(19, 15, str(real_cheaters))
#     sheet.write(19, 16, str(size_trainset))

#     precision7_1 = np.append(precision7_1, precision_score(test_y, pred7))
#     recall7_1 = np.append(recall7_1, recall_score(test_y, pred7))
#     f1_score7_1 = np.append(f1_score7_1, f1_score(test_y, pred7))
#     kappa7_1 = np.append(kappa7_1, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
#     roc7_1 = np.append(roc7_1, auc)
#     fpr7_1 = np.append(fpr7_1, fpr)

#     ##################
#     pred7 = (probs7 >= 0.5).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred7).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs7)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(20, 0, "Random Forest")
#     sheet.write(20, 1, "idur_OEF")
#     sheet.write(20, 2, "0.5")
#     sheet.write(20, 3, precision_score(test_y, pred7))
#     sheet.write(20, 4, recall_score(test_y, pred7))
#     sheet.write(20, 5, f1_score(test_y, pred7))
#     sheet.write(20, 6, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
#     sheet.write(20, 7, auc)
#     sheet.write(20, 8, fpr)
#     sheet.write(20, 9, str(tp))
#     sheet.write(20, 10, str(fn))
#     sheet.write(20, 11, str(fp))
#     sheet.write(20, 12, str(tn))
#     sheet.write(20, 13, str(tot_n))
#     sheet.write(20, 14, str(fptp_rate))
#     sheet.write(20, 15, str(real_cheaters))
#     sheet.write(20, 16, str(size_trainset))

#     precision7_2 = np.append(precision7_2, precision_score(test_y, pred7))
#     recall7_2 = np.append(recall7_2, recall_score(test_y, pred7))
#     f1_score7_2 = np.append(f1_score7_2, f1_score(test_y, pred7))
#     kappa7_2 = np.append(kappa7_2, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
#     roc7_2 = np.append(roc7_2, auc)
#     fpr7_2 = np.append(fpr7_2, fpr)

#     ##################
#     pred7 = (probs7 >= 0.97).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred7).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs7)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(21, 0, "Random Forest")
#     sheet.write(21, 1, "idur_OEF")
#     sheet.write(21, 2, "0.97")
#     sheet.write(21, 3, precision_score(test_y, pred7))
#     sheet.write(21, 4, recall_score(test_y, pred7))
#     sheet.write(21, 5, f1_score(test_y, pred7))
#     sheet.write(21, 6, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
#     sheet.write(21, 7, auc)
#     sheet.write(21, 8, fpr)
#     sheet.write(21, 9, str(tp))
#     sheet.write(21, 10, str(fn))
#     sheet.write(21, 11, str(fp))
#     sheet.write(21, 12, str(tn))
#     sheet.write(21, 13, str(tot_n))
#     sheet.write(21, 14, str(fptp_rate))
#     sheet.write(21, 15, str(real_cheaters))
#     sheet.write(21, 16, str(size_trainset))

#     precision7_3 = np.append(precision7_3, precision_score(test_y, pred7))
#     recall7_3 = np.append(recall7_3, recall_score(test_y, pred7))
#     f1_score7_3 = np.append(f1_score7_3, f1_score(test_y, pred7))
#     kappa7_3 = np.append(kappa7_3, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
#     roc7_3 = np.append(roc7_3, auc)
#     fpr7_3 = np.append(fpr7_3, fpr)

 
# #################################################
#     ###
#     model8 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)
#     # Fit, Predict
#     model8.fit(x_smote, y_smote)
#     probs8 = model8.predict_proba(test_x)
#     probs8 = probs8[:, 1]

#     pred8 = (probs8 >= 0.7).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred8).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs8)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(22, 0, "Gradient Boosting")
#     sheet.write(22, 1, "iraw_OEF")
#     sheet.write(22, 2, "0.7")
#     sheet.write(22, 3, precision_score(test_y, pred8))
#     sheet.write(22, 4, recall_score(test_y, pred8))
#     sheet.write(22, 5, f1_score(test_y, pred8))
#     sheet.write(22, 6, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
#     sheet.write(22, 7, auc)
#     sheet.write(22, 8, fpr)
#     sheet.write(22, 9, str(tp))
#     sheet.write(22, 10, str(fn))
#     sheet.write(22, 11, str(fp))
#     sheet.write(22, 12, str(tn))
#     sheet.write(22, 13, str(tot_n))
#     sheet.write(22, 14, str(fptp_rate))
#     sheet.write(22, 15, str(real_cheaters))
#     sheet.write(22, 16, str(size_trainset))

#     precision8_1 = np.append(precision8_1, precision_score(test_y, pred8))
#     recall8_1 = np.append(recall8_1, recall_score(test_y, pred8))
#     f1_score8_1 = np.append(f1_score8_1, f1_score(test_y, pred8))
#     kappa8_1 = np.append(kappa8_1, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
#     roc8_1 = np.append(roc8_1, auc)
#     fpr8_1 = np.append(fpr8_1, fpr)

#     ######################
#     pred8 = (probs8 >= 0.5).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred8).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs8)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(23, 0, "Gradient Boosting")
#     sheet.write(23, 1, "iraw_OEF")
#     sheet.write(23, 2, "0.5")
#     sheet.write(23, 3, precision_score(test_y, pred8))
#     sheet.write(23, 4, recall_score(test_y, pred8))
#     sheet.write(23, 5, f1_score(test_y, pred8))
#     sheet.write(23, 6, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
#     sheet.write(23, 7, auc)
#     sheet.write(23, 8, fpr)
#     sheet.write(23, 9, str(tp))
#     sheet.write(23, 10, str(fn))
#     sheet.write(23, 11, str(fp))
#     sheet.write(23, 12, str(tn))
#     sheet.write(23, 13, str(tot_n))
#     sheet.write(23, 14, str(fptp_rate))
#     sheet.write(23, 15, str(real_cheaters))
#     sheet.write(23, 16, str(size_trainset))

#     precision8_2 = np.append(precision8_2, precision_score(test_y, pred8))
#     recall8_2 = np.append(recall8_2, recall_score(test_y, pred8))
#     f1_score8_2 = np.append(f1_score8_2, f1_score(test_y, pred8))
#     kappa8_2 = np.append(kappa8_2, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
#     roc8_2 = np.append(roc8_2, auc)
#     fpr8_2 = np.append(fpr8_2, fpr)

#     ######################
#     pred8 = (probs8 >= 0.97).astype(bool) 

#     tn, fp, fn, tp = confusion_matrix(test_y, pred8).ravel()
#     fpr = (fp)/(fp+tn)
#     auc = roc_auc_score(test_y, probs8)
#     fptp_rate = (fp+tp)/(tn+fp+fn+tp)
#     tot_n =  tn+fp+fn+tp
#     real_cheaters = tp+fn

#     #### write to excel sheet ####
#     sheet.write(24, 0, "Gradient Boosting")
#     sheet.write(24, 1, "iraw_OEF")
#     sheet.write(24, 2, "0.97")
#     sheet.write(24, 3, precision_score(test_y, pred8))
#     sheet.write(24, 4, recall_score(test_y, pred8))
#     sheet.write(24, 5, f1_score(test_y, pred8))
#     sheet.write(24, 6, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
#     sheet.write(24, 7, auc)
#     sheet.write(24, 8, fpr)
#     sheet.write(24, 9, str(tp))
#     sheet.write(24, 10, str(fn))
#     sheet.write(24, 11, str(fp))
#     sheet.write(24, 12, str(tn))
#     sheet.write(24, 13, str(tot_n))
#     sheet.write(24, 14, str(fptp_rate))
#     sheet.write(24, 15, str(real_cheaters))
#     sheet.write(24, 16, str(size_trainset))

#     precision8_3 = np.append(precision8_3, precision_score(test_y, pred8))
#     recall8_3 = np.append(recall8_3, recall_score(test_y, pred8))
#     f1_score8_3 = np.append(f1_score8_3, f1_score(test_y, pred8))
#     kappa8_3 = np.append(kappa8_3, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
#     roc8_3 = np.append(roc8_3, auc)
#     fpr8_3 = np.append(fpr8_3, fpr)




  
########### stacking #######################
#### base models ####
    estimators8= [
      DecisionTreeClassifier(criterion="entropy", max_features="auto", splitter="best"),
      SVC(C=2, gamma="auto", kernel="rbf", probability = True),
      GaussianNB(var_smoothing=6.579332246575682e-08),
      MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam"),
      GradientBoostingClassifier(learning_rate=0.1, n_estimators=100),
      LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen"),
      LogisticRegression(C=2, multi_class="multinomial", solver="newton-cg"),
      RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
      ]

    estimators7= [
      DecisionTreeClassifier(criterion="entropy", max_features="auto", splitter="best"),
      GaussianNB(var_smoothing=6.579332246575682e-08),
      MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam"),
      GradientBoostingClassifier(learning_rate=0.1, n_estimators=100),
      LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen"),
      LogisticRegression(C=2, multi_class="multinomial", solver="newton-cg"),
      RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
      ]

    estimators6= [
      GaussianNB(var_smoothing=6.579332246575682e-08),
      MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam"),
      GradientBoostingClassifier(learning_rate=0.1, n_estimators=100),
      LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen"),
      LogisticRegression(C=2, multi_class="multinomial", solver="newton-cg"),
      RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
      ]

    estimators5= [
      MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam"),
      GradientBoostingClassifier(learning_rate=0.1, n_estimators=100),
      LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen"),
      GaussianNB(var_smoothing=6.579332246575682e-08),
      RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
      ]

    estimators4 = [
      MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam"),
      GradientBoostingClassifier(learning_rate=0.1, n_estimators=100),
      GaussianNB(var_smoothing=6.579332246575682e-08),
      RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
      ]

    estimators3 = [
      MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam"),
      GradientBoostingClassifier(learning_rate=0.1, n_estimators=100),
      RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
      ]

    estimators2 = [
      GradientBoostingClassifier(learning_rate=0.1, n_estimators=100),
      RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
      ]

    # sclf8 = StackingClassifier(estimators8, meta_classifier=LogisticRegression() )
    # sclf7 = StackingClassifier(estimators7, meta_classifier=LogisticRegression() )
    # sclf6 = StackingClassifier(estimators6, meta_classifier=LogisticRegression() )
    # sclf5 = StackingClassifier(estimators5, meta_classifier=LogisticRegression() )
    # sclf4 = StackingClassifier(estimators4, meta_classifier=LogisticRegression() )
    # sclf3 = StackingClassifier(estimators3, meta_classifier=LogisticRegression() )
    # sclf2 = StackingClassifier(estimators2, meta_classifier=LogisticRegression() )

    # pipe1 = make_pipeline(FunctionTransformer(feature_1),
    #                       GradientBoostingClassifier(learning_rate=0.1, n_estimators=100))
    # pipe2 = make_pipeline(FunctionTransformer(feature_2),
    #                       RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911))
    # pipe3 = make_pipeline(FunctionTransformer(feature_all),
    #                       MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam"))
    # pipe4 = make_pipeline(FunctionTransformer(feature_all),
    #                       GaussianNB(var_smoothing=6.579332246575682e-08))
    # pipe5 = make_pipeline(FunctionTransformer(feature_all),
    #                       LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen"))
    # pipe6 = make_pipeline(FunctionTransformer(feature_all),
    #                       LogisticRegression(C=2, multi_class="multinomial", solver="newton-cg"))
    # pipe7 = make_pipeline(FunctionTransformer(feature_all),
    #                       DecisionTreeClassifier(criterion="entropy", max_features="auto", splitter="best"))
    # pipe8 = make_pipeline(FunctionTransformer(feature_all),
    #                       SVC(C=2, gamma="auto", kernel="rbf", probability = True))

                          

    # sclf2 = StackingClassifier(classifiers=[pipe1, pipe2], 
    #                           meta_classifier=LogisticRegression())
    # sclf3 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3], 
    #                           meta_classifier=LogisticRegression())
    # sclf4 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4], 
    #                           meta_classifier=LogisticRegression())
    # sclf5 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5], 
    #                           meta_classifier=LogisticRegression())
    # sclf6 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6], 
    #                           meta_classifier=LogisticRegression())      
    # sclf7 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7], 
    #                           meta_classifier=LogisticRegression())
    # sclf8 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe8], 
    #                           meta_classifier=LogisticRegression())
  
    # sclf2 = StackingClassifier(classifiers=[pipe1, pipe2], 
    #                           meta_classifier=SVC(kernel='linear', probability=True))
    # sclf3 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3], 
    #                           meta_classifier=SVC(kernel='linear', probability=True))
    # sclf4 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4], 
    #                           meta_classifier=SVC(kernel='linear', probability=True))
    # sclf5 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5], 
    #                           meta_classifier=SVC(kernel='linear', probability=True))
    # sclf6 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6], 
    #                           meta_classifier=SVC(kernel='linear', probability=True))      
    # sclf7 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7], 
    #                           meta_classifier=SVC(kernel='linear', probability=True))
    # sclf8 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe8], 
    #                           meta_classifier=SVC(kernel='linear', probability=True))


    # sclf2 = StackingClassifier(classifiers=[pipe1, pipe2], 
    #                           meta_classifier=DecisionTreeClassifier())
    # sclf3 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3], 
    #                           meta_classifier=DecisionTreeClassifier())
    # sclf4 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4], 
    #                           meta_classifier=DecisionTreeClassifier())
    # sclf5 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5], 
    #                           meta_classifier=DecisionTreeClassifier())
    # sclf6 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6], 
    #                           meta_classifier=DecisionTreeClassifier())      
    # sclf7 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7], 
    #                           meta_classifier=DecisionTreeClassifier())
    # sclf8 = StackingClassifier(classifiers=[pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe8], 
    #                           meta_classifier=DecisionTreeClassifier())
    
    sclf8 = StackingClassifier(estimators8, meta_classifier=GradientBoostingClassifier() )
    sclf7 = StackingClassifier(estimators7, meta_classifier=GradientBoostingClassifier() )
    sclf6 = StackingClassifier(estimators6, meta_classifier=GradientBoostingClassifier() )
    sclf5 = StackingClassifier(estimators5, meta_classifier=GradientBoostingClassifier() )
    sclf4 = StackingClassifier(estimators4, meta_classifier=GradientBoostingClassifier() )
    sclf3 = StackingClassifier(estimators3, meta_classifier=GradientBoostingClassifier() )
    sclf2 = StackingClassifier(estimators2, meta_classifier=GradientBoostingClassifier() )

    
    sclf2.fit(x_smote, y_smote)
    sclf3.fit(x_smote, y_smote)
    sclf4.fit(x_smote, y_smote)
    sclf5.fit(x_smote, y_smote)
    sclf6.fit(x_smote, y_smote)
    sclf7.fit(x_smote, y_smote)
    sclf8.fit(x_smote, y_smote)


########
    probs_stacking2 = sclf2.predict_proba(test_x)
    probs_stacking2 = probs_stacking2[:, 1]
    pred_stacking2 = (probs_stacking2 >= 0.5).astype(bool) 
    f1_stacking2 = f1_score(test_y, pred_stacking2)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_stacking2).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs_stacking2)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp
    real_cheaters = tp+fn

    #### write to excel sheet ####
    sheet.write(25, 0, "stacking2")
    sheet.write(25, 1, "")
    sheet.write(25, 2, "0.5")
    sheet.write(25, 3, precision_score(test_y, pred_stacking2))
    sheet.write(25, 4, recall_score(test_y, pred_stacking2))
    sheet.write(25, 5, f1_stacking2)
    sheet.write(25, 6, cohen_kappa_score(test_y, pred_stacking2, weights = 'quadratic'))
    sheet.write(25, 7, auc)
    sheet.write(25, 8, fpr)
    sheet.write(25, 9, str(tp))
    sheet.write(25, 10, str(fn))
    sheet.write(25, 11, str(fp))
    sheet.write(25, 12, str(tn))
    sheet.write(25, 13, str(tot_n))
    sheet.write(25, 14, str(fptp_rate))
    sheet.write(25, 15, str(real_cheaters))
    sheet.write(25, 16, str(size_trainset))

    precision_s2 = np.append(precision_s2, precision_score(test_y, pred_stacking2))
    recall_s2 = np.append(recall_s2, recall_score(test_y, pred_stacking2))
    f1_s2 = np.append(f1_s2, f1_stacking2)
    kappa_s2 = np.append(kappa_s2, cohen_kappa_score(test_y, pred_stacking2, weights = 'quadratic'))
    roc_s2 = np.append(roc_s2, auc)
    fpr_s2 = np.append(fpr_s2, fpr)


##########
    probs_stacking3 = sclf3.predict_proba(test_x)
    probs_stacking3 = probs_stacking3[:, 1]
    pred_stacking3 = (probs_stacking3 >= 0.5).astype(bool) 
    f1_stacking3 = f1_score(test_y, pred_stacking3)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_stacking3).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs_stacking3)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp
    real_cheaters = tp+fn

    #### write to excel sheet ####
    sheet.write(26, 0, "stacking3")
    sheet.write(26, 1, "")
    sheet.write(26, 2, "0.5")
    sheet.write(26, 3, precision_score(test_y, pred_stacking3))
    sheet.write(26, 4, recall_score(test_y, pred_stacking3))
    sheet.write(26, 5, f1_stacking3)
    sheet.write(26, 6, cohen_kappa_score(test_y, pred_stacking3, weights = 'quadratic'))
    sheet.write(26, 7, auc)
    sheet.write(26, 8, fpr)
    sheet.write(26, 9, str(tp))
    sheet.write(26, 10, str(fn))
    sheet.write(26, 11, str(fp))
    sheet.write(26, 12, str(tn))
    sheet.write(26, 13, str(tot_n))
    sheet.write(26, 14, str(fptp_rate))
    sheet.write(26, 15, str(real_cheaters))
    sheet.write(26, 16, str(size_trainset))

    precision_s3 = np.append(precision_s3, precision_score(test_y, pred_stacking3))
    recall_s3 = np.append(recall_s3, recall_score(test_y, pred_stacking3))
    f1_s3 = np.append(f1_s3, f1_stacking3)
    kappa_s3 = np.append(kappa_s3, cohen_kappa_score(test_y, pred_stacking3, weights = 'quadratic'))
    roc_s3 = np.append(roc_s3, auc)
    fpr_s3 = np.append(fpr_s3, fpr)

#########
    probs_stacking4 = sclf4.predict_proba(test_x)
    probs_stacking4 = probs_stacking4[:, 1]
    pred_stacking4 = (probs_stacking4 >= 0.5).astype(bool) 
    f1_stacking4 = f1_score(test_y, pred_stacking4)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_stacking4).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs_stacking4)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp
    real_cheaters = tp+fn

    #### write to excel sheet ####
    sheet.write(27, 0, "stacking4")
    sheet.write(27, 1, "")
    sheet.write(27, 2, "0.5")
    sheet.write(27, 3, precision_score(test_y, pred_stacking4))
    sheet.write(27, 4, recall_score(test_y, pred_stacking4))
    sheet.write(27, 5, f1_stacking4)
    sheet.write(27, 6, cohen_kappa_score(test_y, pred_stacking4, weights = 'quadratic'))
    sheet.write(27, 7, auc)
    sheet.write(27, 8, fpr)
    sheet.write(27, 9, str(tp))
    sheet.write(27, 10, str(fn))
    sheet.write(27, 11, str(fp))
    sheet.write(27, 12, str(tn))
    sheet.write(27, 13, str(tot_n))
    sheet.write(27, 14, str(fptp_rate))
    sheet.write(27, 15, str(real_cheaters))
    sheet.write(27, 16, str(size_trainset))

    precision_s4 = np.append(precision_s4, precision_score(test_y, pred_stacking4))
    recall_s4 = np.append(recall_s4, recall_score(test_y, pred_stacking4))
    f1_s4 = np.append(f1_s4, f1_stacking4)
    kappa_s4 = np.append(kappa_s4, cohen_kappa_score(test_y, pred_stacking4, weights = 'quadratic'))
    roc_s4 = np.append(roc_s4, auc)
    fpr_s4 = np.append(fpr_s4, fpr)

#########
    probs_stacking5 = sclf5.predict_proba(test_x)
    probs_stacking5 = probs_stacking5[:, 1]
    pred_stacking5 = (probs_stacking5 >= 0.5).astype(bool) 
    f1_stacking5 = f1_score(test_y, pred_stacking5)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_stacking5).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs_stacking5)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp
    real_cheaters = tp+fn

    #### write to excel sheet ####
    sheet.write(28, 0, "stacking5")
    sheet.write(28, 1, "")
    sheet.write(28, 2, "0.5")
    sheet.write(28, 3, precision_score(test_y, pred_stacking5))
    sheet.write(28, 4, recall_score(test_y, pred_stacking5))
    sheet.write(28, 5, f1_stacking5)
    sheet.write(28, 6, cohen_kappa_score(test_y, pred_stacking5, weights = 'quadratic'))
    sheet.write(28, 7, auc)
    sheet.write(28, 8, fpr)
    sheet.write(28, 9, str(tp))
    sheet.write(28, 10, str(fn))
    sheet.write(28, 11, str(fp))
    sheet.write(28, 12, str(tn))
    sheet.write(28, 13, str(tot_n))
    sheet.write(28, 14, str(fptp_rate))
    sheet.write(28, 15, str(real_cheaters))
    sheet.write(28, 16, str(size_trainset))

    precision_s5 = np.append(precision_s5, precision_score(test_y, pred_stacking5))
    recall_s5 = np.append(recall_s5, recall_score(test_y, pred_stacking5))
    f1_s5 = np.append(f1_s5, f1_stacking5)
    kappa_s5 = np.append(kappa_s5, cohen_kappa_score(test_y, pred_stacking5, weights = 'quadratic'))
    roc_s5 = np.append(roc_s5, auc)
    fpr_s5 = np.append(fpr_s5, fpr)

#########
    probs_stacking6 = sclf6.predict_proba(test_x)
    probs_stacking6 = probs_stacking6[:, 1]
    pred_stacking6 = (probs_stacking6 >= 0.5).astype(bool) 
    f1_stacking6 = f1_score(test_y, pred_stacking6)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_stacking6).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs_stacking6)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp
    real_cheaters = tp+fn

    #### write to excel sheet ####
    sheet.write(29, 0, "stacking6")
    sheet.write(29, 1, "")
    sheet.write(29, 2, "0.5")
    sheet.write(29, 3, precision_score(test_y, pred_stacking6))
    sheet.write(29, 4, recall_score(test_y, pred_stacking6))
    sheet.write(29, 5, f1_stacking6)
    sheet.write(29, 6, cohen_kappa_score(test_y, pred_stacking6, weights = 'quadratic'))
    sheet.write(29, 7, auc)
    sheet.write(29, 8, fpr)
    sheet.write(29, 9, str(tp))
    sheet.write(29, 10, str(fn))
    sheet.write(29, 11, str(fp))
    sheet.write(29, 12, str(tn))
    sheet.write(29, 13, str(tot_n))
    sheet.write(29, 14, str(fptp_rate))
    sheet.write(29, 15, str(real_cheaters))
    sheet.write(29, 16, str(size_trainset))

    precision_s6 = np.append(precision_s6, precision_score(test_y, pred_stacking6))
    recall_s6 = np.append(recall_s6, recall_score(test_y, pred_stacking6))
    f1_s6 = np.append(f1_s6, f1_stacking6)
    kappa_s6 = np.append(kappa_s6, cohen_kappa_score(test_y, pred_stacking6, weights = 'quadratic'))
    roc_s6 = np.append(roc_s6, auc)
    fpr_s6 = np.append(fpr_s6, fpr)

#########
    probs_stacking7 = sclf7.predict_proba(test_x)
    probs_stacking7 = probs_stacking7[:, 1]
    pred_stacking7 = (probs_stacking7 >= 0.5).astype(bool) 
    f1_stacking7 = f1_score(test_y, pred_stacking7)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_stacking7).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs_stacking7)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp
    real_cheaters = tp+fn

    #### write to excel sheet ####
    sheet.write(30, 0, "stacking7")
    sheet.write(30, 1, "")
    sheet.write(30, 2, "0.5")
    sheet.write(30, 3, precision_score(test_y, pred_stacking7))
    sheet.write(30, 4, recall_score(test_y, pred_stacking7))
    sheet.write(30, 5, f1_stacking7)
    sheet.write(30, 6, cohen_kappa_score(test_y, pred_stacking7, weights = 'quadratic'))
    sheet.write(30, 7, auc)
    sheet.write(30, 8, fpr)
    sheet.write(30, 9, str(tp))
    sheet.write(30, 10, str(fn))
    sheet.write(30, 11, str(fp))
    sheet.write(30, 12, str(tn))
    sheet.write(30, 13, str(tot_n))
    sheet.write(30, 14, str(fptp_rate))
    sheet.write(30, 15, str(real_cheaters))
    sheet.write(30, 16, str(size_trainset))

    precision_s7 = np.append(precision_s7, precision_score(test_y, pred_stacking7))
    recall_s7 = np.append(recall_s7, recall_score(test_y, pred_stacking7))
    f1_s7 = np.append(f1_s7, f1_stacking7)
    kappa_s7 = np.append(kappa_s7, cohen_kappa_score(test_y, pred_stacking7, weights = 'quadratic'))
    roc_s7 = np.append(roc_s7, auc)
    fpr_s7 = np.append(fpr_s7, fpr)

#########
    probs_stacking8 = sclf8.predict_proba(test_x)
    probs_stacking8 = probs_stacking8[:, 1]
    pred_stacking8 = (probs_stacking8 >= 0.5).astype(bool) 
    f1_stacking8 = f1_score(test_y, pred_stacking8)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_stacking8).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs_stacking8)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp
    real_cheaters = tp+fn

    #### write to excel sheet ####
    sheet.write(31, 0, "stacking8")
    sheet.write(31, 1, "")
    sheet.write(31, 2, "0.5")
    sheet.write(31, 3, precision_score(test_y, pred_stacking8))
    sheet.write(31, 4, recall_score(test_y, pred_stacking8))
    sheet.write(31, 5, f1_stacking8)
    sheet.write(31, 6, cohen_kappa_score(test_y, pred_stacking8, weights = 'quadratic'))
    sheet.write(31, 7, auc)
    sheet.write(31, 8, fpr)
    sheet.write(31, 9, str(tp))
    sheet.write(31, 10, str(fn))
    sheet.write(31, 11, str(fp))
    sheet.write(31, 12, str(tn))
    sheet.write(31, 13, str(tot_n))
    sheet.write(31, 14, str(fptp_rate))
    sheet.write(31, 15, str(real_cheaters))
    sheet.write(31, 16, str(size_trainset))

    precision_s8 = np.append(precision_s8, precision_score(test_y, pred_stacking8))
    recall_s8 = np.append(recall_s8, recall_score(test_y, pred_stacking8))
    f1_s8 = np.append(f1_s8, f1_stacking8)
    kappa_s8 = np.append(kappa_s8, cohen_kappa_score(test_y, pred_stacking8, weights = 'quadratic'))
    roc_s8 = np.append(roc_s8, auc)
    fpr_s8 = np.append(fpr_s8, fpr)

############
sheet_mean = wb.add_sheet('mean', cell_overwrite_ok=True) 

sheet_mean.write(0, 0, 'Model')
sheet_mean.write(0, 1, 'dataset')
sheet_mean.write(0, 2, 'threshold')
sheet_mean.write(0, 3, 'Precision')
sheet_mean.write(0, 4, 'Recall')
sheet_mean.write(0, 5, 'F1 Score')
sheet_mean.write(0, 6, 'weighted Kappa')
sheet_mean.write(0, 7, 'ROC AUC')
sheet_mean.write(0, 8, 'FPR')

# sheet_mean.write(1, 0, "Decision Tree")
# sheet_mean.write(1, 1, "iraw_idur_OEF")
# sheet_mean.write(1, 2, "0.7")
# sheet_mean.write(1, 3, np.mean(precision1_1))
# sheet_mean.write(1, 4, np.mean(recall1_1))
# sheet_mean.write(1, 5, np.mean(f1_score1_1))
# sheet_mean.write(1, 6, np.mean(kappa1_1))
# sheet_mean.write(1, 7, np.mean(roc1_1))
# sheet_mean.write(1, 8, np.mean(fpr1_1))

# sheet_mean.write(2, 0, "Decision Tree")
# sheet_mean.write(2, 1, "iraw_idur_OEF")
# sheet_mean.write(2, 2, "0.5")
# sheet_mean.write(2, 3, np.mean(precision1_2))
# sheet_mean.write(2, 4, np.mean(recall1_2))
# sheet_mean.write(2, 5, np.mean(f1_score1_2))
# sheet_mean.write(2, 6, np.mean(kappa1_2))
# sheet_mean.write(2, 7, np.mean(roc1_2))
# sheet_mean.write(2, 8, np.mean(fpr1_2))

# sheet_mean.write(3, 0, "Decision Tree")
# sheet_mean.write(3, 1, "iraw_idur_OEF")
# sheet_mean.write(3, 2, "0.97")
# sheet_mean.write(3, 3, np.mean(precision1_3))
# sheet_mean.write(3, 4, np.mean(recall1_3))
# sheet_mean.write(3, 5, np.mean(f1_score1_3))
# sheet_mean.write(3, 6, np.mean(kappa1_3))
# sheet_mean.write(3, 7, np.mean(roc1_3))
# sheet_mean.write(3, 8, np.mean(fpr1_3))


# ###########
# sheet_mean.write(4, 0, "Support Vector Machine")
# sheet_mean.write(4, 1, "iraw_idur_OEF")
# sheet_mean.write(4, 2, "0.7")
# sheet_mean.write(4, 3, np.mean(precision2_1))
# sheet_mean.write(4, 4, np.mean(recall2_1))
# sheet_mean.write(4, 5, np.mean(f1_score2_1))
# sheet_mean.write(4, 6, np.mean(kappa2_1))
# sheet_mean.write(4, 7, np.mean(roc2_1))
# sheet_mean.write(4, 8, np.mean(fpr2_1))

# sheet_mean.write(5, 0, "Support Vector Machine")
# sheet_mean.write(5, 1, "iraw_idur_OEF")
# sheet_mean.write(5, 2, "0.5")
# sheet_mean.write(5, 3, np.mean(precision2_2))
# sheet_mean.write(5, 4, np.mean(recall2_2))
# sheet_mean.write(5, 5, np.mean(f1_score2_2))
# sheet_mean.write(5, 6, np.mean(kappa2_2))
# sheet_mean.write(5, 7, np.mean(roc2_2))
# sheet_mean.write(5, 8, np.mean(fpr2_2))

# sheet_mean.write(6, 0, "Support Vector Machine")
# sheet_mean.write(6, 1, "iraw_idur_OEF")
# sheet_mean.write(6, 2, "0.97")
# sheet_mean.write(6, 3, np.mean(precision2_3))
# sheet_mean.write(6, 4, np.mean(recall2_3))
# sheet_mean.write(6, 5, np.mean(f1_score2_3))
# sheet_mean.write(6, 6, np.mean(kappa2_3))
# sheet_mean.write(6, 7, np.mean(roc2_3))
# sheet_mean.write(6, 8, np.mean(fpr2_3))


# ######################
# sheet_mean.write(7, 0, "Logistic Regression")
# sheet_mean.write(7, 1, "iraw_idur_OEF")
# sheet_mean.write(7, 2, "0.7")
# sheet_mean.write(7, 3, np.mean(precision3_1))
# sheet_mean.write(7, 4, np.mean(recall3_1))
# sheet_mean.write(7, 5, np.mean(f1_score3_1))
# sheet_mean.write(7, 6, np.mean(kappa3_1))
# sheet_mean.write(7, 7, np.mean(roc3_1))
# sheet_mean.write(7, 8, np.mean(fpr3_1))

# sheet_mean.write(8, 0, "Logistic Regression")
# sheet_mean.write(8, 1, "iraw_idur_OEF")
# sheet_mean.write(8, 2, "0.5")
# sheet_mean.write(8, 3, np.mean(precision3_2))
# sheet_mean.write(8, 4, np.mean(recall3_2))
# sheet_mean.write(8, 5, np.mean(f1_score3_2))
# sheet_mean.write(8, 6, np.mean(kappa3_2))
# sheet_mean.write(8, 7, np.mean(roc3_2))
# sheet_mean.write(8, 8, np.mean(fpr3_2))

# sheet_mean.write(9, 0, "Logistic Regression")
# sheet_mean.write(9, 1, "iraw_idur_OEF")
# sheet_mean.write(9, 2, "0.97")
# sheet_mean.write(9, 3, np.mean(precision3_3))
# sheet_mean.write(9, 4, np.mean(recall3_3))
# sheet_mean.write(9, 5, np.mean(f1_score3_3))
# sheet_mean.write(9, 6, np.mean(kappa3_3))
# sheet_mean.write(9, 7, np.mean(roc3_3))
# sheet_mean.write(9, 8, np.mean(fpr3_3))


# ##################################
# sheet_mean.write(10, 0, "Discriminant Analysis")
# sheet_mean.write(10, 1, "iraw_idur_OEF")
# sheet_mean.write(10, 2, "0.7")
# sheet_mean.write(10, 3, np.mean(precision4_1))
# sheet_mean.write(10, 4, np.mean(recall4_1))
# sheet_mean.write(10, 5, np.mean(f1_score4_1))
# sheet_mean.write(10, 6, np.mean(kappa4_1))
# sheet_mean.write(10, 7, np.mean(roc4_1))
# sheet_mean.write(10, 8, np.mean(fpr4_1))

# sheet_mean.write(11, 0, "Discriminant Analysis")
# sheet_mean.write(11, 1, "iraw_idur_OEF")
# sheet_mean.write(11, 2, "0.5")
# sheet_mean.write(11, 3, np.mean(precision4_2))
# sheet_mean.write(11, 4, np.mean(recall4_2))
# sheet_mean.write(11, 5, np.mean(f1_score4_2))
# sheet_mean.write(11, 6, np.mean(kappa4_2))
# sheet_mean.write(11, 7, np.mean(roc4_2))
# sheet_mean.write(11, 8, np.mean(fpr4_2))

# sheet_mean.write(12, 0, "Discriminant Analysis")
# sheet_mean.write(12, 1, "iraw_idur_OEF")
# sheet_mean.write(12, 2, "0.97")
# sheet_mean.write(12, 3, np.mean(precision4_3))
# sheet_mean.write(12, 4, np.mean(recall4_3))
# sheet_mean.write(12, 5, np.mean(f1_score4_3))
# sheet_mean.write(12, 6, np.mean(kappa4_3))
# sheet_mean.write(12, 7, np.mean(roc4_3))
# sheet_mean.write(12, 8, np.mean(fpr4_3))



# #####################################
# sheet_mean.write(13, 0, "Naive Bayes")
# sheet_mean.write(13, 1, "iraw_idur_OEF")
# sheet_mean.write(13, 2, "0.7")
# sheet_mean.write(13, 3, np.mean(precision5_1))
# sheet_mean.write(13, 4, np.mean(recall5_1))
# sheet_mean.write(13, 5, np.mean(f1_score5_1))
# sheet_mean.write(13, 6, np.mean(kappa5_1))
# sheet_mean.write(13, 7, np.mean(roc5_1))
# sheet_mean.write(13, 8, np.mean(fpr5_1))

# sheet_mean.write(14, 0, "Naive Bayes")
# sheet_mean.write(14, 1, "iraw_idur_OEF")
# sheet_mean.write(14, 2, "0.5")
# sheet_mean.write(14, 3, np.mean(precision5_2))
# sheet_mean.write(14, 4, np.mean(recall5_2))
# sheet_mean.write(14, 5, np.mean(f1_score5_2))
# sheet_mean.write(14, 6, np.mean(kappa5_2))
# sheet_mean.write(14, 7, np.mean(roc5_2))
# sheet_mean.write(14, 8, np.mean(fpr5_2))

# sheet_mean.write(15, 0, "Naive Bayes")
# sheet_mean.write(15, 1, "iraw_idur_OEF")
# sheet_mean.write(15, 2, "0.97")
# sheet_mean.write(15, 3, np.mean(precision5_3))
# sheet_mean.write(15, 4, np.mean(recall5_3))
# sheet_mean.write(15, 5, np.mean(f1_score5_3))
# sheet_mean.write(15, 6, np.mean(kappa5_3))
# sheet_mean.write(15, 7, np.mean(roc5_3))
# sheet_mean.write(15, 8, np.mean(fpr5_3))


# #########################################
# sheet_mean.write(16, 0, "Neural Network")
# sheet_mean.write(16, 1, "iraw_idur_OEF")
# sheet_mean.write(16, 2, "0.7")
# sheet_mean.write(16, 3, np.mean(precision6_1))
# sheet_mean.write(16, 4, np.mean(recall6_1))
# sheet_mean.write(16, 5, np.mean(f1_score6_1))
# sheet_mean.write(16, 6, np.mean(kappa6_1))
# sheet_mean.write(16, 7, np.mean(roc6_1))
# sheet_mean.write(16, 8, np.mean(fpr6_1))

# sheet_mean.write(17, 0, "Neural Network")
# sheet_mean.write(17, 1, "iraw_idur_OEF")
# sheet_mean.write(17, 2, "0.5")
# sheet_mean.write(17, 3, np.mean(precision6_2))
# sheet_mean.write(17, 4, np.mean(recall6_2))
# sheet_mean.write(17, 5, np.mean(f1_score6_2))
# sheet_mean.write(17, 6, np.mean(kappa6_2))
# sheet_mean.write(17, 7, np.mean(roc6_2))
# sheet_mean.write(17, 8, np.mean(fpr6_2))

# sheet_mean.write(18, 0, "Neural Network")
# sheet_mean.write(18, 1, "iraw_idur_OEF")
# sheet_mean.write(18, 2, "0.97")
# sheet_mean.write(18, 3, np.mean(precision6_3))
# sheet_mean.write(18, 4, np.mean(recall6_3))
# sheet_mean.write(18, 5, np.mean(f1_score6_3))
# sheet_mean.write(18, 6, np.mean(kappa6_3))
# sheet_mean.write(18, 7, np.mean(roc6_3))
# sheet_mean.write(18, 8, np.mean(fpr6_3))


# #####################################
# sheet_mean.write(19, 0, "Random Forest")
# sheet_mean.write(19, 1, "iraw_idur_OEF")
# sheet_mean.write(19, 2, "0.7")
# sheet_mean.write(19, 3, np.mean(precision7_1))
# sheet_mean.write(19, 4, np.mean(recall7_1))
# sheet_mean.write(19, 5, np.mean(f1_score7_1))
# sheet_mean.write(19, 6, np.mean(kappa7_1))
# sheet_mean.write(19, 7, np.mean(roc7_1))
# sheet_mean.write(19, 8, np.mean(fpr7_1))

# sheet_mean.write(20, 0, "Random Forest")
# sheet_mean.write(20, 1, "iraw_idur_OEF")
# sheet_mean.write(20, 2, "0.5")
# sheet_mean.write(20, 3, np.mean(precision7_2))
# sheet_mean.write(20, 4, np.mean(recall7_2))
# sheet_mean.write(20, 5, np.mean(f1_score7_2))
# sheet_mean.write(20, 6, np.mean(kappa7_2))
# sheet_mean.write(20, 7, np.mean(roc7_2))
# sheet_mean.write(20, 8, np.mean(fpr7_2))

# sheet_mean.write(21, 0, "Random Forest")
# sheet_mean.write(21, 1, "iraw_idur_OEF")
# sheet_mean.write(21, 2, "0.97")
# sheet_mean.write(21, 3, np.mean(precision7_3))
# sheet_mean.write(21, 4, np.mean(recall7_3))
# sheet_mean.write(21, 5, np.mean(f1_score7_3))
# sheet_mean.write(21, 6, np.mean(kappa7_3))
# sheet_mean.write(21, 7, np.mean(roc7_3))
# sheet_mean.write(21, 8, np.mean(fpr7_3))


# ####################################
# sheet_mean.write(22, 0, "Gradient Boosting")
# sheet_mean.write(22, 1, "iraw_idur_OEF")
# sheet_mean.write(22, 2, "0.7")
# sheet_mean.write(22, 3, np.mean(precision8_1))
# sheet_mean.write(22, 4, np.mean(recall8_1))
# sheet_mean.write(22, 5, np.mean(f1_score8_1))
# sheet_mean.write(22, 6, np.mean(kappa8_1))
# sheet_mean.write(22, 7, np.mean(roc8_1))
# sheet_mean.write(22, 8, np.mean(fpr8_1))

# sheet_mean.write(23, 0, "Gradient Boosting")
# sheet_mean.write(23, 1, "iraw_idur_OEF")
# sheet_mean.write(23, 2, "0.5")
# sheet_mean.write(23, 3, np.mean(precision8_2))
# sheet_mean.write(23, 4, np.mean(recall8_2))
# sheet_mean.write(23, 5, np.mean(f1_score8_2))
# sheet_mean.write(23, 6, np.mean(kappa8_2))
# sheet_mean.write(23, 7, np.mean(roc8_2))
# sheet_mean.write(23, 8, np.mean(fpr8_2))

# sheet_mean.write(24, 0, "Gradient Boosting")
# sheet_mean.write(24, 1, "iraw_idur_OEF")
# sheet_mean.write(24, 2, "0.97")
# sheet_mean.write(24, 3, np.mean(precision8_3))
# sheet_mean.write(24, 4, np.mean(recall8_3))
# sheet_mean.write(24, 5, np.mean(f1_score8_3))
# sheet_mean.write(24, 6, np.mean(kappa8_3))
# sheet_mean.write(24, 7, np.mean(roc8_3))
# sheet_mean.write(24, 8, np.mean(fpr8_3))

sheet_mean.write(25, 0, "stacking2")
sheet_mean.write(25, 1, "")
sheet_mean.write(25, 2, "0.5")
sheet_mean.write(25, 3, np.mean(precision_s2))
sheet_mean.write(25, 4, np.mean(recall_s2))
sheet_mean.write(25, 5, np.mean(f1_s2))
sheet_mean.write(25, 6, np.mean(kappa_s2))
sheet_mean.write(25, 7, np.mean(roc_s2))
sheet_mean.write(25, 8, np.mean(fpr_s2))

sheet_mean.write(26, 0, "stacking3")
sheet_mean.write(26, 1, "")
sheet_mean.write(26, 2, "0.5")
sheet_mean.write(26, 3, np.mean(precision_s3))
sheet_mean.write(26, 4, np.mean(recall_s3))
sheet_mean.write(26, 5, np.mean(f1_s3))
sheet_mean.write(26, 6, np.mean(kappa_s3))
sheet_mean.write(26, 7, np.mean(roc_s3))
sheet_mean.write(26, 8, np.mean(fpr_s3))

sheet_mean.write(27, 0, "stacking4")
sheet_mean.write(27, 1, "")
sheet_mean.write(27, 2, "0.5")
sheet_mean.write(27, 3, np.mean(precision_s4))
sheet_mean.write(27, 4, np.mean(recall_s4))
sheet_mean.write(27, 5, np.mean(f1_s4))
sheet_mean.write(27, 6, np.mean(kappa_s4))
sheet_mean.write(27, 7, np.mean(roc_s4))
sheet_mean.write(27, 8, np.mean(fpr_s4))

sheet_mean.write(28, 0, "stacking5")
sheet_mean.write(28, 1, "")
sheet_mean.write(28, 2, "0.5")
sheet_mean.write(28, 3, np.mean(precision_s5))
sheet_mean.write(28, 4, np.mean(recall_s5))
sheet_mean.write(28, 5, np.mean(f1_s5))
sheet_mean.write(28, 6, np.mean(kappa_s5))
sheet_mean.write(28, 7, np.mean(roc_s5))
sheet_mean.write(28, 8, np.mean(fpr_s5))

sheet_mean.write(29, 0, "stacking6")
sheet_mean.write(29, 1, "")
sheet_mean.write(29, 2, "0.5")
sheet_mean.write(29, 3, np.mean(precision_s6))
sheet_mean.write(29, 4, np.mean(recall_s6))
sheet_mean.write(29, 5, np.mean(f1_s6))
sheet_mean.write(29, 6, np.mean(kappa_s6))
sheet_mean.write(29, 7, np.mean(roc_s6))
sheet_mean.write(29, 8, np.mean(fpr_s6))

sheet_mean.write(30, 0, "stacking7")
sheet_mean.write(30, 1, "")
sheet_mean.write(30, 2, "0.5")
sheet_mean.write(30, 3, np.mean(precision_s7))
sheet_mean.write(30, 4, np.mean(recall_s7))
sheet_mean.write(30, 5, np.mean(f1_s7))
sheet_mean.write(30, 6, np.mean(kappa_s7))
sheet_mean.write(30, 7, np.mean(roc_s7))
sheet_mean.write(30, 8, np.mean(fpr_s7))

sheet_mean.write(31, 0, "stacking8")
sheet_mean.write(31, 1, "")
sheet_mean.write(31, 2, "0.5")
sheet_mean.write(31, 3, np.mean(precision_s8))
sheet_mean.write(31, 4, np.mean(recall_s8))
sheet_mean.write(31, 5, np.mean(f1_s8))
sheet_mean.write(31, 6, np.mean(kappa_s8))
sheet_mean.write(31, 7, np.mean(roc_s8))
sheet_mean.write(31, 8, np.mean(fpr_s8))


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

In [28]:
wb.save('1form_both50_04_seed25_top10features_gb.xls') 